**Table of contents**<a id='toc0_'></a>    
- [Menganalisis Risiko Gagal Bayar Peminjam](#toc1_)    
  - [Buka *file* data dan baca informasi umumnya.](#toc1_1_)    
  - [Soal 1. Eksplorasi data](#toc1_2_)    
  - [Transformasi data](#toc1_3_)    
- [Bekerja dengan nilai yang hilang](#toc2_)    
    - [Memperbaiki nilai yang hilang di `total_income`](#toc2_1_1_)    
    - [ Memperbaiki nilai di `days_employed`](#toc2_1_2_)    
  - [Pengkategorian Data](#toc2_2_)    
  - [Memeriksa hipotesis](#toc2_3_)    
- [Kesimpulan umum](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Menganalisis Risiko Gagal Bayar Peminjam](#toc0_)


# Tujuan Proyek
Tujuan dari projek ini adalah untuk membantu Divisi Kredit untuk menentukan kemampuan nasabah dalam menyelesaikan kredit. Parameter historis *gagal bayar* nasabah yang pernah melakukan kredit pada masa sebelumnya akan menjadi informasi dasar sebagai proyeksi untuk calon nasabah ke depannya. Output dari projek ini adalah dapat memberi informasi mengenai kriteria khas dari nasabah yang berpotensi mengalami gagal bayar dan yang melunasi kredit dengan baik.

Adapun hipotesis yang akan diuji pada projek ini adalah sebagai berikut.
1. Jumlah anak yang lebih sedikit dalam keluarga akan meningkatkan kemampuan nasabah dalam melunasi kredit.
2. Nasabah yang belum membina keluarga akan memiliki potensi lebih kecil untuk mengalami gagal bayar dibandingkan nasabah yang telah berkeluarga.
3. Nasabah dengan pendapatan yang lebih kecil memiliki potensi lebih besar untuk gagal bayar.
4. Tujuan atas pengajuan kredit akan menentukan kemampuan pelunasan pinjaman oleh nasabah.


## <a id='toc1_1_'></a>[Buka *file* data dan baca informasi umumnya.](#toc0_)

In [1]:
# Muat semua *library*
import pandas as pd
import numpy as np

# Muat datanya
try:
    scoring = pd.read_csv('../src/DataBase_PROJEK_credit_scoring_eng.csv')
except:
    scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')



In [2]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita
scoring.shape



(21525, 12)

In [3]:
# Mari tampilkan N baris pertama
scoring.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Di atas adalah 15 baris pertama dari *dataset* yang akan kita gunakan. Terlihat terdapat sejumlah masalah dari *dataset* tersebut yang perlu di selesaikan terlebih dahulu sebelum melakukan analisa. Berikut sejumlah masalah yang ditemukan.

* Terdapat nilai minus pada sejumlah *cell* di kolom `'days_employed'` serta memiliki format *float*. Dimana seharusnya variabel `'days_employed'` harus bernilai positif.
* Terdapat penulisan variabel kategorik yang beragam pada kolom `'education' `untuk *id* yang sama. Terdapat sejumlah variasi penulisan `secondary education` dengan id bernilai 1 pada kolom tersebut seperti penggunaan huruf kapital, *uppercase*, *lowercase*, dll.
* Terdapat sejumlah variabel kategorik yang berbeda untuk makna yang sama pada kolom `'purpose'`. Contohnya seperti kalimat 'to have a wedding' dan 'having a wedding' yang sama-sama memiliki arti pinjaman untuk pernikahan.
* Terdapat sejumlah data yang hilang.

In [4]:
# Melihat informasi data
scoring.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Dari informasi data tersebut dapat dilihat kolom `'days_employed'` dan `'total_income'` memiliki nilai yang hilang. Dan ternyata jumlah data yang hilang pada kedua kolom tersebut juga berjumlah sama.

In [5]:
# Mari kita lihat tabel yang telah difilter dengan nilai yang hilang di kolom pertama yang mengandung data yang hilang

scoring.loc[scoring['days_employed'].isna()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


Setelah melihat tabel di atas yang berisi baris dengan kolom `'days_employed'` bernilai `NaN` terlihat pola simetris pada 15 baris pertama yang ditampilkan. Hal tersebut membawa kita untuk berasumsi bila kekosongan nilai pada kolom `'days_employed'` berkorelasi dengan hilangnya nilai pada kolom `'total_income'`. Perlu dilakukan penyelidikan lebih lanjut untuk memastikan jumlah baris untuk nilai yang hilang pada kedua kolom tersebut bernilai sama dengan total baris yang hilang pada *dataset*.

In [6]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.

scoring_nan = scoring.loc[scoring['days_employed'].isna()&scoring['total_income'].isna()] # dataframe untuk baris dengan nilai NaN

scoring_nan.shape[0]


2174

**Kesimpulan sementara**

Dari informasi data yang kita peroleh sebelumnya diketahui bahwa nilai `Non-null` untuk kolom `'days_employed'` dan `'total_income'` yaitu sebanyak 19351 baris untuk masing-masing kolom. Sedangkan ukuran dataset secara keseluruhan adalah sebanyak 21525 baris sehingga total baris yang memiliki nilai hilang untuk masing-masing kolom adalah 2174 baris. Sedangkan untuk baris dengan kolom `'days_employed'` dan `'total_income'` bernilai `NaN` juga diperoleh total baris sebanyak 2174. Sehingga dapat disimpulkan bahwa setiap baris dengan nilai yang hilang pada kolom `'days_employed'` secara bersamaan juga memiliki nilai yang hilang pada kolom `'total_income'`.


In [7]:
## Menghitung persentase baris dengan data yang hilang.
print(f"Persentase nilai yang hilang adalah {scoring_nan.shape[0]/scoring.shape[0]:.1%} dari data keseluruhan")

Persentase nilai yang hilang adalah 10.1% dari data keseluruhan


Adapun persentase baris yang hilang terhadap keseluruhan baris pada dataset yaitu sebesar 10.1%. Persentase nilai yang hilang ini cukup besar sehingga mengisi nilai yang hilang menjadi pilihan dari pada menghapus seluruh data tersebut. Untuk dapat mengisi nilai yang hilang tersebut kita perlu mengetahui apakah sejumlah data yang hilang berasal dari karakteristik nasabah tertentu. Selain itu, kita perlu memeriksa apakah ada ketergantungan nilai antara kolom dengan data yang hilang dengan kolom lainnya yang dapat mengidentifikasi karakteristik nasabah.

Saat ini kita memiliki 12 kolom yang menggambarkan karakteristik setiap nasabah. Selain itu kita juga memiliki sejumlah data yang memiliki 2 karakteristik yang hilang secara bersamaan yaitu `'days_employed'` dan `'total_income'`. Dari 10 data karakteristik lainnya yang tersisa, data `'income_type'` dan `'dob_years'` memiliki hubungan yang paling erat atas parameter total pendapatan bulanan serta berapa lama nasabah telah bekerja.

Sekarang kita akan memeriksa kelompok *unique* `'income_type'` dan `'dob_years'` apa saja yang terdapat pada data keseluruhan dan data yang memiliki nilai `NaN`.

In [8]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang

## Melihat unique_value pada kolom 'income_type' pada baris yang mempunyai nilai NaN
print(scoring_nan['income_type'].sort_values().unique())

print()
## Melihat unique_value pada kolom 'income_type' pada seluruh baris dataset
print(scoring['income_type'].sort_values().unique())



['business' 'civil servant' 'employee' 'entrepreneur' 'retiree']

['business' 'civil servant' 'employee' 'entrepreneur'
 'paternity / maternity leave' 'retiree' 'student' 'unemployed']


In [9]:
# Memeriksa distribusinya

## Membuat tabel distribusi untuk data keseluruhan dan data dengan nilai yang hilang
income_type_unique = scoring['income_type'].unique()

income_type_dist = pd.DataFrame({
    'unique':income_type_unique,
    'dataset_count':[pd.Series(filter(lambda x:x==item, scoring['income_type'])).count() for item in income_type_unique],
    'nan_dataset_count':[pd.Series(filter(lambda x:x==item, scoring_nan['income_type'])).count() for item in income_type_unique]
    })

income_type_dist['dataset_count'] = income_type_dist['dataset_count'].replace(0,np.nan)
income_type_dist['nan_dataset_count'] = income_type_dist['nan_dataset_count'].replace(0,np.nan)
income_type_dist['distribution_ratio'] = income_type_dist['nan_dataset_count']/income_type_dist['dataset_count']
income_type_dist['dataset_contribution'] = income_type_dist['dataset_count']/scoring.shape[0]
income_type_dist['nan_dataset_contribution'] = income_type_dist['nan_dataset_count']/scoring.shape[0]

income_type_dist.sort_values(by='nan_dataset_count', ascending=False).reset_index(drop=True)

,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,employee,11119,1105.0,0.099379,0.516562,0.051336
1,business,5085,508.0,0.099902,0.236237,0.023600
2,retiree,3856,413.0,0.107106,0.179141,0.019187
3,civil servant,1459,147.0,0.100754,0.067782,0.006829
4,entrepreneur,2,1.0,0.500000,0.000093,0.000046
5,unemployed,2,NaN,NaN,0.000093,NaN
6,student,1,NaN,NaN,0.000046,NaN
7,paternity / maternity leave,1,NaN,NaN,0.000046,NaN


In [10]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang

## Melihat unique_value pada kolom 'dob_years' pada baris yang mempunyai nilai NaN
print(scoring_nan['dob_years'].sort_values().unique())

print()
## Melihat unique_value pada kolom 'dob_years' pada seluruh baris dataset
print(scoring['dob_years'].sort_values().unique())

[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73]

[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]


In [11]:
# Periksalah distribusinya

## Membuat tabel distribusi untuk data keseluruhan dan data dengan nilai yang hilang
dob_years_unique = scoring['dob_years'].unique()

dob_years_dist = pd.DataFrame({
    'unique':dob_years_unique,
    'dataset_count':[pd.Series(filter(lambda x:x==item, scoring['dob_years'])).count() for item in dob_years_unique],
    'nan_dataset_count':[pd.Series(filter(lambda x:x==item, scoring_nan['dob_years'])).count() for item in dob_years_unique]
    })

dob_years_dist['dataset_count'] = dob_years_dist['dataset_count'].replace(0,np.nan)
dob_years_dist['nan_dataset_count'] = dob_years_dist['nan_dataset_count'].replace(0,np.nan)
dob_years_dist['distribution_ratio'] = dob_years_dist['nan_dataset_count']/dob_years_dist['dataset_count']
dob_years_dist['dataset_contribution'] = dob_years_dist['dataset_count']/scoring.shape[0]
dob_years_dist['nan_dataset_contribution'] = dob_years_dist['nan_dataset_count']/scoring.shape[0]

dob_years_dist.sort_values(by='nan_dataset_count', ascending=False).reset_index(drop=True)

,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,34,603,69.0,0.114428,0.028014,0.003206
1,40,609,66.0,0.108374,0.028293,0.003066
2,42,597,65.0,0.108878,0.027735,0.003020
3,31,560,65.0,0.116071,0.026016,0.003020
4,35,617,64.0,0.103728,0.028664,0.002973
5,36,555,63.0,0.113514,0.025784,0.002927
6,47,480,59.0,0.122917,0.022300,0.002741
7,41,607,59.0,0.097199,0.028200,0.002741
8,30,540,58.0,0.107407,0.025087,0.002695
9,28,503,57.0,0.113320,0.023368,0.002648


**1. Karakteristik Jenis Pekerjaan (`'income_type'`)**

Pada data keseluruhan ditemukan sebanyak 8 kelompok karakteristik jenis pekerjaan. Sedangkan pada data yang memiliki nilai hilang hanya ditemukan 5 kelompok karakteristik jenis pekerjaan. 3 kelompok karakteristik yang tidak masuk ke dalam data dengan kolom tak bernilai yaitu `'paternity / maternity leave'`, `'student'` dan  `'unemployed'`

Kelompok `'employee'`, `'business'`, `'retiree'`, dan `'civil servant'` merupakan kelompok nasabah teratas yang memiliki jumlah terbanyak baik pada data dengan nilai yang hilang ataupun pada data keseluruhan. Adapun dari 10.1% data yang hilang, kelompok ini memiliki kontribusi sebesar 5.13%, 2.36%, 1.92% dan 0.68% secara berurutan. Dan berdasarkan `'distribution_ratio'` untuk keempat kelompok pekerjaan tersebut juga memiliki nilai yang seragam yaitu berkisar pada nilai 10%.

Untuk kelompok `'entrepreneur'` memiliki `'distribution_ratio'` yang sangat besar yaitu 50%, akan tetapi jika dilihat pada keseluruhan dataset, kelompok ini hanya berkontribusi sebesar 0.01% dari keseluruhan nasabah.

**2. Karakteristik Usia Nasabah (`'dob_years'`)**

Pada data keseluruhan ditemukan sebanyak 58 kelompok karakteristik usia nasabah. Sedangkan pada data yang memiliki nilai hilang memiliki 56 kelompok karakteristik. Adapun dua kelompok umur yang tidak terdapat pada data yang hilang yaitu kelompok usia 74 dan 75 tahun.

Bila dilihat kelompok usia yang memiliki kontribusi data yang hilang (`'nan_dataset_contribution'`) lebih besar dari 0.25% maka akan ditemukan sebanyak 15 nasabah dengan beragam usia dengan `'distribution_ratio'` pada rentang 9% hingga 12%. Adapun usia paling kecil yaitu 28 tahun sedangkan yang paling tua adalah 58 tahun. Apabila 15 nasabah ini diakumulasikan kontribusi data yang hilang-nya (`'nan_dataset_contribution'`) maka akan diperoleh sebesar 4.2% dari 10.1% data yang hilang.

**Kemungkinan penyebab hilangnya nilai dalam data**

In [12]:
## Melihat median dan mean untuk kelompok jenis pekerjaan
income_type_dist.agg({'distribution_ratio':['mean','median']})

,distribution_ratio
mean,0.181428
median,0.100754


In [13]:
## Melihat median dan mean untuk kelompok usia
dob_years_dist.agg({'distribution_ratio':['mean','median']})

,distribution_ratio
mean,0.099629
median,0.099913


kelompok `'employee'`, `'business'`, `'retiree'`, dan `'civil servant'` merupakan kelompok mayoritas pada *dataset* ini. Apabila total nasabah dikelompok ini dijumlahkan maka akan diperoleh sebanyak 21519 nasabah atau sama dengan 99.97% dari keseluruhan *dataset*. Selain itu dari distribusi kelompok yang telah dilakukan juga ditemukan bahwa kelompok ini juga mayoritas sebagai kelompok yang mengisi data secara lengkap sekaligus kelompok yang mengisi data tidak lengkap. Kemudian dari `'distribution_ratio'` juga terlihat nilai rasio yang seragam untuk setiap kelompok ini yaitu pada rentang 10% hingga 11%.

Selain itu, pada pengelompokan berdasarkan usia nasabah juga ditemukan `'distribution_ratio'` yang cukup seragam untuk rentang umur yang sangat besar. Hal tersebut terlihat dari nilai median dan mean `'distribution_ratio'` yang memiliki nilai sama besar.

Oleh karena itu kesimpulan sementara saya adalah nilai-nilai yang hilang/`NaN` pada *dataset* diakibatkan oleh `randomness` karena memiliki persebaran rasio distribusi yang sama untuk setiap kelompok mayoritas.

Oleh karena itu kita akan mencoba untuk eksplorasi *dataset* ini berdasarkan karakteristik lainnya untuk melihat apakah terdapat pola yang mengindikasikan alasan hilangnya nilai tersebut. Berikut dibawah ini akan dilakukan pengelompokan data berdasarkan kolom karakteristik yang kita miliki kemudian diamati distribusinya.

**Memeriksa apakah nilai yang hilang bersifat acak atau tidak.**

In [14]:
# Memeriksa distribusi di seluruh *dataset*

## Membuat tabel distribusi untuk data keseluruhan dan data dengan nilai yang hilang
## Karakteristik Jumlah Anak (Kolom 'children')
children_unique = scoring['children'].unique()

children_dist = pd.DataFrame({
    'unique':children_unique,
    'dataset_count':[pd.Series(filter(lambda x:x==item, scoring['children'])).count() for item in children_unique],
    'nan_dataset_count':[pd.Series(filter(lambda x:x==item, scoring_nan['children'])).count() for item in children_unique]
    })

children_dist['dataset_count'] = children_dist['dataset_count'].replace(0,np.nan)
children_dist['nan_dataset_count'] = children_dist['nan_dataset_count'].replace(0,np.nan)
children_dist['distribution_ratio'] = children_dist['nan_dataset_count']/children_dist['dataset_count']
children_dist['dataset_contribution'] = children_dist['dataset_count']/scoring.shape[0]
children_dist['nan_dataset_contribution'] = children_dist['nan_dataset_count']/scoring.shape[0]

children_dist.sort_values(by='nan_dataset_count', ascending=False).reset_index(drop=True)

,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,0,14149,1439,0.101703,0.657329,0.066852
1,1,4818,475,0.098589,0.223833,0.022067
2,2,2055,204,0.099270,0.095470,0.009477
3,3,330,36,0.109091,0.015331,0.001672
4,20,76,9,0.118421,0.003531,0.000418
5,4,41,7,0.170732,0.001905,0.000325
6,-1,47,3,0.063830,0.002184,0.000139
7,5,9,1,0.111111,0.000418,0.000046


In [15]:
# Memeriksa distribusi di seluruh *dataset*

## Membuat tabel distribusi untuk data keseluruhan dan data dengan nilai yang hilang
## Karakteristik Tingkat Pendidikan Nasabah (Kolom 'education_id')
education_id_unique = scoring['education_id'].unique()

education_id_dist = pd.DataFrame({
    'unique':education_id_unique,
    'dataset_count':[pd.Series(filter(lambda x:x==item, scoring['education_id'])).count() for item in education_id_unique],
    'nan_dataset_count':[pd.Series(filter(lambda x:x==item, scoring_nan['education_id'])).count() for item in education_id_unique]
    })

education_id_dist['dataset_count'] = education_id_dist['dataset_count'].replace(0,np.nan)
education_id_dist['nan_dataset_count'] = education_id_dist['nan_dataset_count'].replace(0,np.nan)
education_id_dist['distribution_ratio'] = education_id_dist['nan_dataset_count']/education_id_dist['dataset_count']
education_id_dist['dataset_contribution'] = education_id_dist['dataset_count']/scoring.shape[0]
education_id_dist['nan_dataset_contribution'] = education_id_dist['nan_dataset_count']/scoring.shape[0]

print(scoring.drop_duplicates(subset='education_id')[['education','education_id',]].reset_index(drop=True))
education_id_dist.sort_values(by='nan_dataset_count', ascending=False).reset_index(drop=True)

             education  education_id
0    bachelor's degree             0
1  secondary education             1
2         some college             2
3    primary education             3
4      Graduate Degree             4


,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,1,15233,1540.0,0.101096,0.707689,0.071545
1,0,5260,544.0,0.103422,0.244367,0.025273
2,2,744,69.0,0.092742,0.034564,0.003206
3,3,282,21.0,0.074468,0.013101,0.000976
4,4,6,NaN,NaN,0.000279,NaN


In [16]:
# Memeriksa distribusi di seluruh *dataset*

## Membuat tabel distribusi untuk data keseluruhan dan data dengan nilai yang hilang
## Karakteristik Status Perkawinan (Kolom 'family_status_id')
family_status_id_unique = scoring['family_status_id'].unique()

family_status_id_dist = pd.DataFrame({
    'unique':family_status_id_unique,
    'dataset_count':[pd.Series(filter(lambda x:x==item, scoring['family_status_id'])).count() for item in family_status_id_unique],
    'nan_dataset_count':[pd.Series(filter(lambda x:x==item, scoring_nan['family_status_id'])).count() for item in family_status_id_unique]
    })

family_status_id_dist['dataset_count'] = family_status_id_dist['dataset_count'].replace(0,np.nan)
family_status_id_dist['nan_dataset_count'] = family_status_id_dist['nan_dataset_count'].replace(0,np.nan)
family_status_id_dist['distribution_ratio'] = family_status_id_dist['nan_dataset_count']/family_status_id_dist['dataset_count']
family_status_id_dist['dataset_contribution'] = family_status_id_dist['dataset_count']/scoring.shape[0]
family_status_id_dist['nan_dataset_contribution'] = family_status_id_dist['nan_dataset_count']/scoring.shape[0]

print(scoring.drop_duplicates(subset='family_status_id')[['family_status','family_status_id',]].reset_index(drop=True))
family_status_id_dist.sort_values(by='nan_dataset_count', ascending=False).reset_index(drop=True)

       family_status  family_status_id
0            married                 0
1  civil partnership                 1
2    widow / widower                 2
3           divorced                 3
4          unmarried                 4


,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,0,12380,1237,0.099919,0.575145,0.057468
1,1,4177,442,0.105818,0.194053,0.020534
2,4,2813,288,0.102382,0.130685,0.013380
3,3,1195,112,0.093724,0.055517,0.005203
4,2,960,95,0.098958,0.044599,0.004413


In [17]:
# Memeriksa distribusi di seluruh *dataset*

## Membuat tabel distribusi untuk data keseluruhan dan data dengan nilai yang hilang
## Karakteristik Gender (Kolom 'gender')
gender_unique = scoring['gender'].unique()

gender_dist = pd.DataFrame({
    'unique':gender_unique,
    'dataset_count':[pd.Series(filter(lambda x:x==item, scoring['gender'])).count() for item in gender_unique],
    'nan_dataset_count':[pd.Series(filter(lambda x:x==item, scoring_nan['gender'])).count() for item in gender_unique]
    })

gender_dist['dataset_count'] = gender_dist['dataset_count'].replace(0,np.nan)
gender_dist['nan_dataset_count'] = gender_dist['nan_dataset_count'].replace(0,np.nan)
gender_dist['distribution_ratio'] = gender_dist['nan_dataset_count']/gender_dist['dataset_count']
gender_dist['dataset_contribution'] = gender_dist['dataset_count']/scoring.shape[0]
gender_dist['nan_dataset_contribution'] = gender_dist['nan_dataset_count']/scoring.shape[0]

gender_dist.sort_values(by='nan_dataset_count', ascending=False).reset_index(drop=True)

,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,F,14236,1484.0,0.104243,0.661370,0.068943
1,M,7288,690.0,0.094676,0.338583,0.032056
2,XNA,1,NaN,NaN,0.000046,NaN


In [18]:
# Memeriksa distribusi di seluruh *dataset*

## Membuat tabel distribusi untuk data keseluruhan dan data dengan nilai yang hilang
## Karakteristik Status Kredit (Kolom 'debt')
debt_unique = scoring['debt'].unique()

debt_dist = pd.DataFrame({
    'unique':debt_unique,
    'dataset_count':[pd.Series(filter(lambda x:x==item, scoring['debt'])).count() for item in debt_unique],
    'nan_dataset_count':[pd.Series(filter(lambda x:x==item, scoring_nan['debt'])).count() for item in debt_unique]
    })

debt_dist['dataset_count'] = debt_dist['dataset_count'].replace(0,np.nan)
debt_dist['nan_dataset_count'] = debt_dist['nan_dataset_count'].replace(0,np.nan)
debt_dist['distribution_ratio'] = debt_dist['nan_dataset_count']/debt_dist['dataset_count']
debt_dist['dataset_contribution'] = debt_dist['dataset_count']/scoring.shape[0]
debt_dist['nan_dataset_contribution'] = debt_dist['nan_dataset_count']/scoring.shape[0]

debt_dist.sort_values(by='nan_dataset_count', ascending=False).reset_index(drop=True)

,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,0,19784,2004,0.101294,0.919117,0.093101
1,1,1741,170,0.097645,0.080883,0.007898


**Kesimpulan sementara**

Setelah dilakukan pengelompokan untuk setiap karakteristik nasabah pada data yang memiliki nilai yang hilang serta data keseluruhan, diperoleh distribusi yang seragam. Hal tersebut tercermin pada hampir keseluruhan data pada setiap kelompok pada setiap karakteristik menunjukkan `'distribution_ratio'` memiliki nilai berkisar pada 10%.

> Sehingga kesimpulan sementara saat ini masih sama yaitu data yang hilang pada *dataset* diakibatkan perilaku acak dari setiap kelompok nasabah. Dimana terdapat 10% nasabah yang memiliki sikap enggan mengisi data-data tersebut dan tersebar secara merata ke setiap karakteristik.


**Penelusuran lanjutan pada data yang hilang**

Kita akan memeriksa apakah terdapat data duplikat pada keseluruhan *dataset*. Kita akan melihat bagaimana persentase kontribusinya serta *distribution ratio*-nya. Ada kemungkinan nasabah yang mengosongkan nilai tersebut melakukan input ulang dan mengisi kembali data-data tersebut dengan lengkap. Cara pengecekan duplikat akan dilakukan pada *subset* kolom `'children'`, `'dob_years'`, `'education_id'`, `'family_status_id'`, `'gender'`, `'income_type'` dan `'debt'`. Alasan pemilihan kolom tersebut sebagai gambaran karakteristik nasabah adalah untuk meminimalisir kesalahan informasi akibat salah dalam pengetikan sehingga kita mengambil data yang menggunakan input berupa angka dan inisial untuk `'gender'`. Sedangkan pemilihan kolom `'income_type'` tetap digunakan karena sebelumnya kita telah mengecek `unique_value` yang dimilikinya tidak terdapat kesalahan pengejaan.

> Dibawah ini ditampilkan jumlah baris yang memiliki data `'children'`, `'dob_years'`, `'education_id'`, `'family_status_id'`, `'gender'`, `'income_type'` dan `'debt'` bernilai sama dengan baris lainnya pada *dataset* keseluruhan kemudian dipilah data yang memiliki nilai `NaN`. Harapannya bila ditemukan jumlah baris sebanyak **2174** maka dapat disimpulkan bila baris data yang hilang merupakan hasil dari duplikat data pada *dataset* keseluruhan.

In [19]:
# Memeriksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang

## Menampilkan jumlah duplikat data pada subset 'children', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type' dan 'debt'
scoring[scoring.duplicated(subset=['children', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type','debt'])].isna().sum()['days_employed']

1587

**Kesimpulan sementara**

Setelah dilakukan dengan pengecekan duplikat data ternyata hanya ditemukan sebanyak 1587 baris data yang sesuai dengan pola yang kita pikirkan. Hal tersebut berarti pola yang kita asumsikan hanya sebanyak 73% dari baris data yang hilang. Angka yang cukup besar akan tetapi tidak dapat dikatakan 100% penyebab data yang hilang terkait suatu pola tertentu. Selain itu tetap ada kemungkinan memang ada sejumlah nasabah yang memiliki data yang sama pada kolom `'children'`, `'dob_years'`, `'education_id'`, `'family_status_id'`, `'gender'`, `'income_type'` dan `'debt'`. Sehingga kesimpulan dari pola ini tidak valid sebagai penyebab data yang hilang.


**Kesimpulan Akhir**

Dari penelusuran yang telah dilakukan atas sejumlah baris dengan data yang hilang, tidak ditemukan karakteristik nasabah tertentu yang menjadi penyebab hilangnya data-data tersebut. Akan tetapi pola yang ditemukan adalah distribusi dari setiap kelompok pada setiap karakteristik menunjukkan pola yang sama antara *dataset* keseluruhan dan data dengan nilai yang hilang. Selain itu, secara keseluruhan setiap kelompok pada setiap karakteristik memiliki *distribution ratio* yang seragam yaitu berada pada kisaran angka 10%. Hal ini menunjukkan kehilangan data sebanyak 10% dari keseluruhan *dataset* diakibatkan oleh hilangnya data sebanyak 10% dari setiap kelompok pada setiap karakteristik (*uniform*).


**Rencana Mengatasi Data yang Hilang**

Dari analisa sejauh ini yang telah dilakukan kita telah ketahui bahwa data yang hilang pada dataset ini hanya ditemukan pada kolom `'days_employed'` dan `'total_income'`. Selain itu juga tidak ditemukan karakteristik nasabah tertentu yang menjadi penyebab kehilangan data ini. Satu-satunya pola yang ditemukan adalah keseragaman distribusi antara data keseluruhan dan data yang hilang.

Kedua kolom `'days_employed'` dan `'total_income'` merupakan kategori kolom dengan variabel kuantitatif. Untuk mengisi data yang hilang maka kita perlu menggunakan perhitungan menggunakan `mean` atau `median`. Akan tetapi untuk melakukannya perlu dilakukan pengelompokan yang tepat. kolom `'days_employed'` menunjukkan berapa lama nasabah telah bekerja dimana hal tersebut berkaitan dengan usia nasabah (`'dob_years'`). Adapun  kolom `'total_income'` menunjukkan pendapatan bulanan dari nasabah dimana hal tersebut berkaitan dengan jenis pekerjaan nasabah (`'income_type'`), berapa lama nasabah telah bekerja (`'days_employed'`), dan tingkat pendidikan nasabah (`'education_id'`).

Oleh karena itu penyelesaian data yang hilang akan dimulai pada kolom `'days_employed'` terlebih dahulu dengan mengelompokkannya dengan parameter yang berkaitan. Kemudian dilanjutkan dengan kolom `'total_income'` dengan melakukan sejumlah pengelompokan pada parameter terkait.


**Rencana Transformasi Data**

Proses selanjutnya yang akan dilakukan adalah melakukan transformasi pada data yang kita miliki. Akan terdapat sejumlah masalah yang mungkin terjadi seperti duplikasi data, penulisan/pengejaan yang kurang tepat, penyeragaman penulisan variabel kategorik, tipe data yang tidak sesuai, nilai data yang keluar dari *constraint* yang seharusnya (nilai tidak wajar), dan penanganan nilai yang hilang.


## <a id='toc1_3_'></a>[Transformasi data](#toc0_)


Proses transformasi akan dimulai dengan memeriksa setiap kolom untuk melihat masalah apa saja yang terjadi pada kolom tersebut. Proses akan diawali dengan memeriksa kolom `'education'` untuk memeriksa apakah terdapat kesalahan penulisan dan menyeragamkan penulisan dalam format *lower case*.


In [20]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
scoring['education'].unique()


array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [21]:
# Memperbaiki penulisan agar menjadi lower_case
scoring['education'] = scoring['education'].str.lower()


In [22]:
# Memeriksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat
scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Setelah memeriksa dan memperbaiki kolom `'education'`, maka dilanjutkan dengan memeriksa kolom `'children'`. Proses akan diawali dengan melihat distribusi setiap kelompok jumlah anak nasabah kemudian dilakukan pengecekan untuk setiap *unique value*,.


In [23]:
# Mari kita lihat distribusi nilai pada kolom `children`
scoring['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Terdapat kelompok jumlah anak nasabah yang tidak wajar pada *dataset* ini. Pertama, terdapat kelompok umur -1 dimana seharusnya umur berformat positif. Kemudian terdapat 76 data nasabah yang memiliki 20 orang anak dimana ini patut dipertanyakan.

Untuk jumlah anak sebesar -1 hal tersebut mungkin dapat terjadi diakibatkan kesalahan dalam penginputannilai untuk jumlah anak sebanyak 1 orang anak. Sedangkan jumlah anak berjumlah 20 orang anak juga perlu dipertanyakan. Apabila seorang ibu melahirkan bukan anak kembar atau bukan melalui jalur adopsi maka seharusnya dibutuhkan minimal 20 tahun untuk memiliki anak sebanyak ini. Hal tersebut tidak wajar dan tercermin pada distribusi penurunan jumlah nasabah seiring meningkatnya jumlah anak. Sehingga nasabah dengan kepemilikan 20 anak juga berkemungkinan adalah keslahan penginputan nilai untuk 2 orang anak.

In [24]:
# Mengganti kolom 'children' bernilai -1 menjadi 1
scoring.loc[scoring['children']==-1, 'children'] = 1

# Mengganti kolom 'children' bernilai 20 menjadi 2
scoring.loc[scoring['children']==20, 'children'] = 2


In [25]:
# Memeriksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki
scoring['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Setelah memperbaiki kolom `'children'` maka dilanjutkan dengan pengecekan kolom `'days_employed'`. Kolom ini memiliki input berupa variabel kuantitatif jumlah hari dan tidak boleh bernilai negatif. Kemudian nilai pada kolom ini bila di convert dalam satuan tahun harus lebih kecil dari usia nasabah.


In [26]:
# Menghitung persentase data dengan kolom `days_employed` yang bernilai negatif
print(f"Persentase data dengan kolom 'days_employed' yang bernilai negatif: {(scoring[scoring['days_employed']<0]['days_employed'].count()/scoring.shape[0]):.2%}")

# Menghitung persentase data kolom `days_employed` yang melebihi usia nasabah bila di konversi ke dalam tahun
print(f"Persentase data kolom 'days_employed' yang melebihi usia nasabah bila di konversi ke dalam tahun: {(scoring[scoring['days_employed']/365 > scoring['dob_years']]['dob_years'].count()/scoring.shape[0]):.2%}")



Persentase data dengan kolom 'days_employed' yang bernilai negatif: 73.90%
Persentase data kolom 'days_employed' yang melebihi usia nasabah bila di konversi ke dalam tahun: 16.00%


Setelah dikalkulasikan, ternyata data nasabah dengan kolom `'days_employed'` bernilai negatif terdapat sebanyak 73.90% dari keseluruhan *dataset*. Sedangkan data nasabah dengan nilai kolom `'days_employed'` dalam format tahun yang melebihi usia nasabah terdapat sebanyak 16.00%. Untuk permasalahan nilai negatif, hal itu mungkin disebabkan oleh kesalahan penginputan seperti kesalahan yang terjadi pada kolom `'children'` sehingga kita akan mengambil nilai positifnya saja untuk perbaikan data. Sedangkan untuk data dengan nilai melebihi usia nasabah, kemungkinan terjadi akibat karakteristik tertentu sehingga akan diperiksa terlebih dahulu.


In [27]:
# Mengganti nilai kolom 'days_employed' bernilai negatif agar menjadi positif
scoring['days_employed'] = scoring['days_employed'].abs()


In [28]:
# Memeriksa hasilnya untuk memastikan bahwa masalahnya telah diperbaiki

## Menghitung persentase data dengan kolom `days_employed` yang bernilai negatif
print(f"Persentase data dengan kolom 'days_employed' yang bernilai negatif: {(scoring[scoring['days_employed']<0]['days_employed'].count()/scoring.shape[0]):.2%}")

## Menghitung persentase data kolom `days_employed` yang melebihi usia nasabah bila di konversi ke dalam tahun
print(f"Persentase data kolom 'days_employed' yang melebihi usia nasabah bila di konversi ke dalam tahun: {(scoring[scoring['days_employed']/365 > scoring['dob_years']]['dob_years'].count()/scoring.shape[0]):.2%}")



Persentase data dengan kolom 'days_employed' yang bernilai negatif: 0.00%
Persentase data kolom 'days_employed' yang melebihi usia nasabah bila di konversi ke dalam tahun: 16.35%


Setelah nilai pada kolom `'days_employed'` dikonversi agar menjadi nilai positif seluruhnya terlihat bahwa terjadi peningkatan persentase nasabah yang memiliki nilai kolom `'days_employed'` melebihi usianya menjadi 16,35% dari yang semula sebesar 16,00%. Selanjutnya kita akan memeriksa kelompok pekerjaan apa saja yang mengakibatkan nilai kolom ini melebihi usia mereka masing-masing.


In [29]:
## Memeriksa distribusion ratio kolom 'income_type' dengan nilai kolom 'days_employed' melebihi usia mereka
print((scoring[scoring['days_employed']/365 > scoring['dob_years']].value_counts(subset='income_type')/scoring.value_counts(subset='income_type')).sort_values(ascending=False))

income_type_dist

income_type
unemployed                     1.000000
retiree                        0.892894
employee                       0.004497
civil servant                  0.004112
business                       0.003540
entrepreneur                        NaN
paternity / maternity leave         NaN
student                             NaN
dtype: float64


,unique,dataset_count,nan_dataset_count,distribution_ratio,dataset_contribution,nan_dataset_contribution
0,employee,11119,1105.0,0.099379,0.516562,0.051336
1,retiree,3856,413.0,0.107106,0.179141,0.019187
2,business,5085,508.0,0.099902,0.236237,0.023600
3,civil servant,1459,147.0,0.100754,0.067782,0.006829
4,unemployed,2,NaN,NaN,0.000093,NaN
5,entrepreneur,2,1.0,0.500000,0.000093,0.000046
6,student,1,NaN,NaN,0.000046,NaN
7,paternity / maternity leave,1,NaN,NaN,0.000046,NaN


In [30]:
## Nilai minimum di kolom 'days_employed'
scoring['days_employed'].min()

24.14163324048118

In [31]:
## Melihat unique_value kolom 'dob_years'
scoring['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

Setelah dilakukan perhitungan *distribution ratio* untuk nasabah dengan durasi kerja melebihi umur mereka masing-masing, terdapat 5 kelompok pekerjaan yang ditemukan. Dari kelima pekerjaan tersebut terdapat 2 kelompok pekerjaan yang dominan yaitu `'unemployed'` dan `'retiree'` dengan *distribution ratio* sebesar 100% dan 89,3% secara berurutan. Sedangkan 3 kelompok pekerjaan lainnya yaitu `'employee'`, `'civil servant'`, dan `'business'` memiliki *distribution ratio* kurang dari 0,45%. 

Karena kolom `'days_employed'` akan digunakan untuk menghitung nilai pengganti untuk data yang hilang pada kolom tersebut serta akan digunakan sebagai acuan pengisian data yang hilang pada kolom `'total_income'`, maka seluruh data yang memiliki nilai `'days_employed'` melebihi usia mereka dan nilai umur nasabah (`'dob_years'`) bukan `0` akan diatur agar memiliki nilai `0` pada kolom `'days_employed'`. Alasan penggunaan angka `0` adalah untuk penanda kolom `'days_employed'` dengan nilai tidak wajar. Dan kita ketahui nilai minimum pada kolom ini yaitu 24.14 dan angka `0` akan menjamin tidak adanya nilai negatif serta melebihi batas usia.

In [32]:
## Mengatur nilai kolom menjadi NaN untuk baris dengan nilai kolom 'days_employed' melebihi usia nasabah
scoring.loc[(scoring['days_employed']/365>scoring['dob_years'])&(scoring['dob_years']!=0),'days_employed'] = 0

In [33]:
# Memeriksa hasilnya untuk memastikan bahwa masalahnya telah diperbaiki

## Menghitung persentase data dengan kolom `days_employed` yang bernilai negatif
print(f"Persentase data dengan kolom 'days_employed' yang bernilai negatif: {(scoring[scoring['days_employed']<0]['days_employed'].count()/scoring.shape[0]):.2%}")

## Menghitung persentase data kolom `days_employed` yang melebihi usia nasabah bila di konversi ke dalam tahun
print(f"Persentase data kolom 'days_employed' yang melebihi usia nasabah bila di konversi ke dalam tahun: {(scoring[scoring['days_employed']/365 > scoring['dob_years']]['dob_years'].count()/scoring.shape[0]):.2%}")



Persentase data dengan kolom 'days_employed' yang bernilai negatif: 0.00%
Persentase data kolom 'days_employed' yang melebihi usia nasabah bila di konversi ke dalam tahun: 0.42%



Ternyata masih terdapat 0.42% data yang memiliki nilai `'days_employed'` melebihi usia mereka akan tetapi itu dikarenakan data umur mereka bernilai `0`.



Setelah memperbaiki data pada kolom `'days_employed'`, kita akan memeriksa data pada kolom `'dob_years'` yang berisi data umur setiap nasabah. Tentunya umur nasabah harus bernilai positif harus cukup dewasa agar dapat mengajukan kredit di bank. Kita asumsikan anak dengan umur 18 tahun ke atas sudah cukup dewasa dan diperbolehkan untuk pengajuan kredit.


In [34]:
# Memeriksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya

## Melihat unique_value kolom 'dob_years'
print(scoring['dob_years'].sort_values().unique())

## Menghitung persentase nasabah berumur kurang dari 18 tahun
print()
print(f"Persentase nasabah berumur kurang dari 18 tahun: {(scoring[scoring['dob_years']<18]['dob_years'].count()/scoring.shape[0]):.2%}")

[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]

Persentase nasabah berumur kurang dari 18 tahun: 0.47%


In [35]:
## Melihat tabel nasabah berumur kurang dari 18 tahun
scoring[scoring['dob_years']<18].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car


Terlihat terdapat data yang aneh disana. Pada baris pertama data yang telah difilter ini tertera nilai `'days_employed'` sebesar 346541.62 hari yang setara dengan 949 tahun, sangat tidak masuk akal. Kita akan memeriksa 25 nilai `'days_employed'` terbesar pada data yang telah difilter ini.


In [36]:
## Menampilkan 'days_employed' dalam format tahun
(scoring['days_employed']/365).sort_values(ascending=False).head(25)

14514    1098.609249
578      1090.017986
16861    1084.640585
14659    1082.053000
10188    1018.261037
1175     1016.108241
1898     1014.094622
8061     1003.994172
7034     1002.925427
15886     974.913109
12729     972.903359
99        949.429093
20462     928.040736
4922      921.961660
12062     910.096862
20577     908.880196
19116     908.379592
16335      50.380685
4299       48.261817
7329       45.461569
17838      44.560821
16825      44.163528
3974       43.385550
1539       43.248435
4321       43.213867
Name: days_employed, dtype: float64

Terdapat 17 baris nilai `'days_employed'` dalam format tahun yang tidak wajar dimulai dari 1096.6 tahun hingga 908.4 tahun. Oleh karena untuk baris dengan nilai `'days_employed'` lebih dari 100 tahun maka nilai kolom `'days_employed'` akan diganti dengan kode angka bernilai `1` dan nilai kolom `'dob_years'` akan diganti dengan kode angka bernilai `100`. Alasan baris tersebut tidak dihapus adalah agar dapat menggunakan informasi di kolom lainnya.


In [37]:
## Mengganti nilai pada abris 'days_employed' > 100 tahun
scoring.loc[scoring['days_employed']/365>100,['days_employed','dob_years']] = [1,100]
## Mengecek apakah 'days_employed' lebih dari 100 tahun telah diperbaiki
scoring[scoring['days_employed']/365>100]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [38]:
## Persentase data dengan nilai 'dob_years' kecil dari 18 tahun
print(f"[UPDATE] Persentase data dengan nilai 'dob_years' kecil dari 18 tahun: {scoring[scoring['dob_years']<18]['dob_years'].count()/scoring.shape[0]:.2%}")

## Melihat kembali data dengan nilai 'dob_years' kecil dari 18 tahun
scoring[scoring['dob_years']<18].sort_values(by='days_employed', ascending=False)

[UPDATE] Persentase data dengan nilai 'dob_years' kecil dari 18 tahun: 0.39%


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2082,0,10689.250498,0,secondary education,1,married,0,F,employee,0,61819.782,real estate transactions
18851,0,10637.001569,0,secondary education,1,married,0,F,civil servant,0,21845.941,supplementary education
4930,2,8035.487285,0,bachelor's degree,0,divorced,3,F,civil servant,0,18025.839,buy residential real estate
15891,0,6683.979251,0,secondary education,1,married,0,F,employee,0,12850.549,car
10306,0,5944.466834,0,bachelor's degree,0,married,0,F,employee,0,16501.875,buy commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
6670,0,NaN,0,bachelor's degree,0,divorced,3,F,retiree,0,NaN,buy residential real estate
8574,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,property
12403,3,NaN,0,secondary education,1,married,0,M,employee,0,NaN,transactions with commercial real estate
13741,0,NaN,0,secondary education,1,civil partnership,1,F,employee,0,NaN,having a wedding


Sekarang tersisa data dengan `'dob_years'` bernilai `0` tetapi memiliki nilai kolom `'days_employed'` bernilai wajar atau bernilai `Nan`. Sangat susah untuk menentukan umur seseorang bila hanya berpatokan pada nilai `'days_employed'` dan jenis pekerjaannya. Sehingga nilai kolom umur dari data filter yang telah tersisa akan diinput dengan kode angka bernilai `200`.


In [39]:
## Mengatur kolom 'dob_years' bernilai kurang dari 18 tahun agar bernilai 200
scoring.loc[scoring['dob_years']<18,'dob_years'] = 200

In [40]:
# Memeriksa hasilnya untuk memastikan bahwa masalahnya telah diperbaiki

## Memeriksa unique_value untuk kolom umur
scoring['dob_years'].sort_values().unique()


array([ 19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75, 100, 200], dtype=int64)

Sejauh ini kita telah mentransformasikan data pada kolom `'education'`, `'children'`, `'days_employed'`, dan `'dob_years'`. Sekarang kita akan memeriksa data pada kolom `'family_status'`.


In [41]:
# Mari kita lihat nilai untuk kolom ini
scoring['family_status'].unique()


array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

Tidak terdapat masalah pada nilai-nilai di kolom `'family_status'`.


Sekarang kita akan memeriksa nilai pada kolom `'gender'` dan melihat apakah terdapat nilai bermasalah pada kolom tersebut.



In [42]:
# Mari kita liat nilai dalam kolom ini
print(scoring['gender'].unique())

## Memeriksa distribusi-nya
print()
scoring['gender'].value_counts()

['F' 'M' 'XNA']



F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [43]:
## Kita akan menghapus baris dengan nilai kolom 'gender' sama dengan 'XNA'
scoring = scoring.drop(scoring[scoring['gender']=='XNA'].index)


In [44]:
# Memeriksa hasilnya untuk memastikan bahwa masalahnya telah diperbaiki

## Memeriksa distribusi-nya
scoring['gender'].value_counts()


F    14236
M     7288
Name: gender, dtype: int64

Setelah menghapus baris dengan nilai pada kolom `'gender'` yang bernilai salah, sekarang kita akan memeriksa nilai pada kolom `'income_type'`.


In [45]:
# Mari kita lihat nilai dalam kolom ini
print(scoring['income_type'].sort_values().unique())

## Melihat distribusi-nya
print()
scoring['income_type'].value_counts()

['business' 'civil servant' 'employee' 'entrepreneur'
 'paternity / maternity leave' 'retiree' 'student' 'unemployed']



employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Terdapat kelompok pada kolom ini yang bernilai `'entrepreneur'` tetapi hanya memiliki 2 data saja pada *dataset* ini. Dan karena `'entrepreneur'` pada dasarnya serupa dengan kelompok `'business'` yang sama-sama bermakna wirausahawan maka kita kan mengelompokkannya menjadi satu dengan nama `'business'`.


In [46]:
# Mengganti penamaan 'entrepreneur' menjadi 'business'
scoring['income_type'] = scoring['income_type'].replace('entrepreneur', 'business')

In [47]:
# Memeriksa hasilnya untuk memastikan bahwa masalahnya telah diperbaiki
print(scoring['income_type'].sort_values().unique())

## Melihat distribusi-nya
print()
scoring['income_type'].value_counts()


['business' 'civil servant' 'employee' 'paternity / maternity leave'
 'retiree' 'student' 'unemployed']



employee                       11119
business                        5086
retiree                         3856
civil servant                   1459
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

**Memeriksa Data Duplikat**

In [48]:
# Periksa duplikat

## Menghitung jumlah data yang memiliki duplikat
print(scoring.duplicated().sum())

## Menampilkan tabel yang memiliki duplikat
print()
scoring[scoring.duplicated()].sort_values('days_employed')


71



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate


Setelah ditampilkan data yang memiliki duplikat, diperoleh sebanyak 71 data yang ditemukan. Dan ternyata, 71 data yang merupakan duplikat merupakan data yang bernilai `NaN` pada kolom `'days_employed'` dan `'total_income'`. Adapun tindakan yang akan diambil atas data-data duplikat ini yaitu dengan mengeluarkannya pada *dataset* ini.


In [49]:
# Mengatasi duplikat
scoring = scoring.drop_duplicates()

In [50]:
# Melakukan pemeriksaan terakhir untuk mengecek apakah kita memiliki duplikat
print(scoring.duplicated().sum())


0


In [51]:
# Memeriksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang Anda lakukan
scoring.shape

(21453, 12)

**Hasil Transformasi Sementara**

Setelah dilakukan sejumlah transformasi yang telah dilakukan tersisa sebanyak 21453 baris data saat ini. Terjadi pengurangan data sebanyak 72 data atau sebesar 0,33% data awal. Selain menghapus sejumlah baris pada *dataset* awal, kita juga melakukan sejumlah transformasi seperti menggabungkan suatu kelompok ke kelompok lainnya serta memberikan kode nomor khusus untuk kondisi data tertentu. Berikut kode angka yang kita gunakan untuk mentransformasi nilai pada tabel.
* Kolom `'days_employed'` bernilai `0` : Seluruh data yang memiliki nilai `'days_employed'` melebihi usia mereka dan nilai umur nasabah (`'dob_years'`) bukan `0` pada *dataset* awal.
* Kolom `'days_employed'` bernilai `1` dan kolom `'dob_years'` bernilai `100` : Seluruh data yang memiliki nilai `'days_employed'` lebih dari 100 tahun pada *dataset* awal.
* Kolom '`dob_years'` bernilai `200` : Seluruh data yang memiliki nilai `'dob_years'` bernilai `0` tetapi `'days_employed'` kurang dari 100 tahun pada *dataset* awal.


# <a id='toc2_'></a>[Bekerja dengan nilai yang hilang](#toc0_)

Untuk mempercepat proses pengolahan data, maka data yang menggunakan kolom id dan keterangannya akan dipisah. Sehingga pada data log, kita akan menggunakan data ID nya saja. Hal itu dikarenakan pemerosesan data angka jauh lebih cepat dibandingkan proses data string yang tentu terdiri dari sejumlah karakter. Adapun data yang akan kita pisah dan dibuatkan *dictionary* yaitu kolom `'education'` dan `'education_id'` serta kolom `'family_status'` dan `'family_status_id'`.


In [52]:
## Membuat dictionary 'education'
education_ref = scoring[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

## Membuat dictionary 'family_status'
fam_ref = scoring[['family_status_id','family_status']].drop_duplicates().reset_index(drop=True)

## Menghilongkan kolom 'education' dan 'family_status' pada data utama
scoring = scoring.drop(columns=['family_status','education'])
scoring

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,1,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,1,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,1,0,M,employee,0,42820.568,supplementary education
4,0,0.000000,53,1,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,1,1,F,business,0,35966.698,housing transactions
21521,0,0.000000,67,1,0,F,retiree,0,24959.969,purchase of a car
21522,1,2113.346888,38,1,1,M,employee,1,14347.610,property
21523,3,3112.481705,38,1,0,M,employee,1,39054.888,buying my own car


### <a id='toc2_1_1_'></a>[Memperbaiki nilai yang hilang di `total_income`](#toc0_)

Terdapat 2 kolom yang memiliki nilai yang hilang yakni `'total_income'` dan `'days_employed'`. Penanganan nilai yang hilang akan dimulai dari `'total_income'` terlebih dahulu dengan cara membuat kategori usia dengan rentang tertentu pada setiap nasabah. Hal ini bertujuan untuk dapat menghitung total nilai pendapatan nasabah. Adapun pembagian kelompok umur yaitu sebagai berikut.
* Kelompok **under 21**
* Kelompok **21-25**
* Kelompok **26-30**
* Kelompok **31-35**
* Kelompok **36-40**
* Kelompok **41-45**
* Kelompok **46-50**
* Kelompok **51-55**
* Kelompok **56-60**
* Kelompok **61-65**
* Kelompok **66-70**
* Kelompok **71-75**
* Kelompok **problem** : Data bermasalah yaitu kelompok umur 100 dan 200


In [53]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia
def age_category(age_series):
    if age_series<21:
        return "under 21"
    elif (age_series>=21) & (age_series<=25):
        return "21-25"
    elif (age_series>=26) & (age_series<=30):
        return "26-30"
    elif (age_series>=31) & (age_series<=35):
        return "31-35"
    elif (age_series>=36) & (age_series<=40):
        return "36-40"
    elif (age_series>=41) & (age_series<=45):
        return "41-45"
    elif (age_series>=46) & (age_series<=50):
        return "46-50"
    elif (age_series>=51) & (age_series<=55):
        return "51-55"
    elif (age_series>=56) & (age_series<=60):
        return "56-60"
    elif (age_series>=61) & (age_series<=65):
        return "61-65"
    elif (age_series>=66) & (age_series<=70):
        return "66-70"
    elif (age_series>=71) & (age_series<=75):
        return "71-75"
    else:
        return "problem"


In [54]:
# Melakukan pengujian untuk melihat apakah fungsi Anda bekerja atau tidak
age = pd.DataFrame({'dob_years':[19,24, 100, 59]})
age['category'] = age['dob_years'].apply(age_category)
age

,dob_years,category
0,19,under 21
1,24,21-25
2,100,problem
3,59,56-60


In [55]:
# Membuat kolom baru untuk pengategorian usia
scoring['age_category'] = scoring['dob_years'].apply(age_category)


In [56]:
# Memeriksa bagaimana nilai di dalam kolom baru
scoring.head(10)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,0,0,F,employee,0,40620.102,purchase of the house,41-45
1,1,4024.803754,36,1,0,F,employee,0,17932.802,car purchase,36-40
2,0,5623.422610,33,1,0,M,employee,0,23341.752,purchase of the house,31-35
3,3,4124.747207,32,1,0,M,employee,0,42820.568,supplementary education,31-35
4,0,0.000000,53,1,1,F,retiree,0,25378.572,to have a wedding,51-55
5,0,926.185831,27,0,1,M,business,0,40922.170,purchase of the house,26-30
6,0,2879.202052,43,0,0,F,business,0,38484.156,housing transactions,41-45
7,0,152.779569,50,1,0,M,employee,0,21731.829,education,46-50
8,2,6929.865299,35,0,1,F,employee,0,15337.093,having a wedding,31-35
9,0,2188.756445,41,1,0,M,employee,0,23108.150,purchase of the house for my family,41-45


Selain faktor usia, faktor jenis pekerjaan dan berapa lama seseorang telah bekerja pada pekerjaan tersebut juga akan menentukan besaran nilai pendapatan bulanan. Kedua faktor tersebut berperan dalam menentukan seberapa berat jenis pekerjaan dan seberapa besar tingkat keahlian seseorang. Semakin berat dan semakin piawai seseorang dalam melakukan pekerjaan maka mereka layak dibayar lebih mahal. Selain itu untuk beberapa jenis pekerjaan, faktor pendidikan juga menjadi penentu seberapa besar pendapatan mereka karena berkaitan tingkat pengetahuan yang mereka miliki. Sehingga semakin tinggi tingkat pendidikan seseorang pada suatu jenis pekerjaan maka ia akan memiliki pendapatan yang tinggi pula. 



**Membuat tabel yang hanya memuat data tanpa nilai yang hilang**
<br>Data ini akan digunakan untuk memperbaiki nilai yang hilang.

In [57]:
# Membuat tabel tanpa nilai yang hilang
scoring_non_nan = scoring.dropna()
scoring_non_nan.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,0,0,F,employee,0,40620.102,purchase of the house,41-45
1,1,4024.803754,36,1,0,F,employee,0,17932.802,car purchase,36-40
2,0,5623.422610,33,1,0,M,employee,0,23341.752,purchase of the house,31-35
3,3,4124.747207,32,1,0,M,employee,0,42820.568,supplementary education,31-35
4,0,0.000000,53,1,1,F,retiree,0,25378.572,to have a wedding,51-55


In [58]:
# Memperhatikan nilai rata-rata dan median untuk pendapatan berdasarkan faktor yang identifikasi

## Faktor Kelompok Usia
age_category_income = scoring_non_nan.groupby('age_category').agg({'total_income':['mean','median']})
age_category_income['abs_rel_error'] = ((age_category_income['total_income','mean']-age_category_income['total_income','median'])/age_category_income['total_income','mean']).abs()
age_category_income



total_income             abs_rel_error
                      mean      median              
age_category                                        
21-25         23664.156716  21588.0300      0.087733
26-30         26984.340749  23685.3220      0.122257
31-35         28162.233342  24723.3750      0.122109
36-40         28591.236955  24913.8410      0.128620
41-45         28758.450184  24575.3910      0.145455
46-50         27984.178759  24563.6500      0.122231
51-55         26156.618683  22339.7660      0.145923
56-60         24784.377997  21640.4880      0.126850
61-65         24197.711072  20824.6610      0.139395
66-70         21025.052485  17915.9340      0.147877
71-75         19575.454327  18611.5935      0.049238
problem       25334.072890  24387.0700      0.037381
under 21      19586.303559  17257.2770      0.118911

In [59]:
# Memperhatikan nilai rata-rata dan median untuk pendapatan berdasarkan faktor yang identifikasi

## Faktor Jenis Pekerjaan
income_type_income = scoring_non_nan.groupby('income_type').agg({'total_income':['mean','median']})
income_type_income['abs_rel_error'] = ((income_type_income['total_income','mean']-income_type_income['total_income','median'])/income_type_income['total_income','mean']).abs()
income_type_income

total_income             abs_rel_error
                                     mean      median              
income_type                                                        
business                     32397.115286  27577.2720      0.148774
civil servant                27343.729582  24071.6695      0.119664
employee                     25820.841683  22815.1035      0.116407
paternity / maternity leave   8612.661000   8612.6610      0.000000
retiree                      21940.394503  18962.3180      0.135735
student                      15712.260000  15712.2600      0.000000
unemployed                   21014.360500  21014.3605      0.000000

In [60]:
# Memperhatikan nilai rata-rata dan median untuk pendapatan berdasarkan faktor yang identifikasi

## Faktor Tingkat Pendidikan
education_id_income = scoring_non_nan.groupby('education_id').agg({'total_income':['mean','median']})
education_id_income['abs_rel_error'] = ((education_id_income['total_income','mean']-education_id_income['total_income','median'])/education_id_income['total_income','mean']).abs()
education_id_income

total_income             abs_rel_error
                      mean      median              
education_id                                        
0             33142.802434  28054.5310      0.153526
1             24594.503037  21836.5830      0.112136
2             29040.132990  25608.7945      0.118158
3             21144.882211  18741.9760      0.113640
4             27960.024667  25161.5835      0.100087

In [61]:
# Memperhatikan nilai rata-rata dan median untuk pendapatan berdasarkan faktor yang identifikasi

## Faktor Gabungan : Tingkat Pendidikan dan umur
incomeType_ageCategory_income = scoring_non_nan.groupby(['income_type','age_category']).agg({'total_income':['mean','median']})
incomeType_ageCategory_income['abs_rel_error'] = ((incomeType_ageCategory_income['total_income','mean']-incomeType_ageCategory_income['total_income','median'])/incomeType_ageCategory_income['total_income','mean']).abs()
incomeType_ageCategory_income

total_income              \
                                                  mean      median   
income_type                 age_category                             
business                    21-25         26086.131833  23000.3145   
                            26-30         31320.945501  27215.0030   
                            31-35         32402.593403  28506.0405   
                            36-40         33724.640458  29154.2140   
                            41-45         34144.824473  27239.7710   
                            46-50         33794.772672  29495.7840   
                            51-55         32576.323732  26782.3215   
                            56-60         32185.082478  28084.0450   
                            61-65         32265.154202  29127.8650   
                            66-70         34889.489056  29344.2055   
                            71-75         29211.780833  28138.8950   
                            problem       29711.347389  30532.7365   
                            under 21      20223.495136  19648.2745   
civil servant               21-25         24673.419079  23164.8340   
                            26-30         25397.998932  23397.3240   
                            31-35         29292.861643  25813.0010   
                            36-40         27218.528770  23683.4620   
                            41-45         27932.938963  23202.1090   
                            46-50         28160.143630  24115.3055   
                            51-55         26931.066805  23068.1320   
                            56-60         25584.875494  24235.8560   
                            61-65         30040.675405  23066.1870   
                            66-70         34267.048000  27772.0260   
                            71-75         19530.677500  19530.6775   
                            problem       23515.725333  20090.8960   
                            under 21      21099.632667  12125.9860   
employee                    21-25         22412.671190  20780.2140   
                            26-30         25198.304909  22410.9560   
                            31-35         26182.310870  23234.2170   
                            36-40         26515.461959  23363.6440   
                            41-45         26503.753748  23108.1500   
                            46-50         25532.879525  22832.5140   
                            51-55         25722.176337  22379.7350   
                            56-60         27016.773065  23604.9580   
                            61-65         28142.767199  23365.6140   
                            66-70         26457.866857  24881.4565   
                            71-75         23320.184000  20246.1230   
                            problem       25056.023180  23198.6255   
                            under 21      19040.474088  16356.9390   
paternity / maternity leave 36-40          8612.661000   8612.6610   
retiree                     21-25         14298.976000  14298.9760   
                            26-30         14986.931167  11619.3835   
                            31-35         22009.240000  19546.0750   
                            36-40         22216.792037  18454.4280   
                            41-45         27167.409636  22095.7080   
                            46-50         26276.201314  23419.4930   
                            51-55         22071.361547  18919.5865   
                            56-60         21934.500510  19402.2740   
                            61-65         22389.215561  19323.8260   
                            66-70         19834.171420  16963.6290   
                            71-75         18730.020326  18146.7015   
                            problem       22158.874647  16419.4720   
student                     21-25         15712.260000  15712.2600   
unemployed                  31-35          9593.119000   9593.1190   
                            41-45         32435.6020

In [62]:
# Memperhatikan nilai rata-rata dan median untuk pendapatan berdasarkan faktor yang identifikasi

## Faktor Gabungan : Tingkat Pendidikan, umur, dan pendidikan
incomeType_ageCategory_eduId_income = scoring_non_nan.groupby(['income_type','age_category','education_id']).agg({'total_income':['mean','median']})
incomeType_ageCategory_eduId_income['abs_rel_error'] = ((incomeType_ageCategory_eduId_income['total_income','mean']-incomeType_ageCategory_eduId_income['total_income','median'])/incomeType_ageCategory_eduId_income['total_income','mean']).abs()
incomeType_ageCategory_eduId_income.head(20)

total_income             abs_rel_error
                                               mean      median              
income_type age_category education_id                                        
business    21-25        0             28298.294354  25231.3630      0.108379
                         1             24335.526842  21879.7720      0.100912
                         2             26586.259070  22709.1490      0.145831
                         3             19891.302500  19891.3025      0.000000
            26-30        0             35516.950943  31184.4260      0.121985
                         1             27876.560367  25055.4330      0.101201
                         2             29484.323060  28421.4290      0.036049
                         3             29007.883667  21167.1430      0.270297
            31-35        0             38595.400451  32529.3200      0.157171
                         1             28185.561827  26013.5815      0.077060
                         2             29293.287125  28429.5910      0.029484
                         3             22516.703400  16892.3140      0.249787
            36-40        0             39468.034357  33551.2290      0.149914
                         1             30334.972758  26665.1020      0.120978
                         2             36399.607421  33660.9115      0.075240
                         3             28090.882833  27908.0720      0.006508
            41-45        0             43566.794426  34481.8650      0.208529
                         1             29604.400483  25530.7185      0.137604
                         2             44009.622350  38299.5565      0.129746
                         3             29937.792333  31199.2540      0.042136

In [63]:
## Melihat statistik absolute relative error untuk pengelompokkan kategori usia
age_category_income['abs_rel_error'].agg(['mean','median','min','max','var'])

mean      0.114921
median    0.122257
min       0.037381
max       0.147877
var       0.001262
Name: abs_rel_error, dtype: float64

In [64]:
## Melihat statistik absolute relative error untuk pengelompokkan kategori jenis pekerjaan
income_type_income['abs_rel_error'].agg(['mean','median','min','max','var'])

mean      0.074369
median    0.116407
min       0.000000
max       0.148774
var       0.004952
Name: abs_rel_error, dtype: float64

In [65]:
## Melihat statistik absolute relative error untuk pengelompokkan kategori tingkat pendidikan
education_id_income['abs_rel_error'].agg(['mean','median','min','max','var'])

mean      0.119509
median    0.113640
min       0.100087
max       0.153526
var       0.000406
Name: abs_rel_error, dtype: float64

In [66]:
## Melihat statistik absolute relative error untuk pengelompokkan kategori gabungan: jenis pekerjaan+kategori usia
incomeType_ageCategory_income['abs_rel_error'].agg(['mean','median','min','max','var'])

mean      0.118035
median    0.126285
min       0.000000
max       0.425299
var       0.005658
Name: abs_rel_error, dtype: float64

In [67]:
## Melihat statistik absolute relative error untuk pengelompokkan kategori gabungan: jenis pekerjaan+kategori usia+tingkat pendidikan
incomeType_ageCategory_eduId_income['abs_rel_error'].agg(['mean','median','min','max','var'])

mean      0.091078
median    0.098077
min       0.000000
max       0.396689
var       0.005423
Name: abs_rel_error, dtype: float64

Kita telah melakukan sejumlah pengelompokan data untuk dihitung besaran rata-rata, median, serta persentase antara selisih median dan rata-rata terhadap rata-rata. Terdapat 5 pengelompokan yang telah dilakukan yaitu pengelompokan berdasarkan kategori usia, jenis pendapatan, tingkat pendidikan, gabungan (usia dan pekerjaan), serta gabungan (usia, pekerjaan, dan pendidikan). Penggunaan satu kategori saja untuk mengisi data yang hilang menurut saya akan memberikan informasi dengan spektrum yang terlalu luas dan tidak spesifik sehingga itu menjadi alasan dilakukannya pengelompokan secara gabungan. Akan tetapi konsekuensi dari penggabungan kelompok akan dapat mengakibatkan nilai rentang antara minimum dan maksimum *relative error* semakin membesar. Hal tersebut dapat terjadi disebabkan *outlier* dapat berperan secara signifikan di suatu kelompok karena jumlah kelompok makin banyak dan ukuran tiap kelompok akan mengecil. Oleh karena itu parameter *variance relative error* akan menjadi acuan untuk memastikan sedikit/banyaknya kelompok yang memiliki *outlier*.

Bila harus dipilih satu kelompok saja untuk mengganti nilai *total income* yang hilang maka saya akan memilih pengelompokan berdasarkan jenis pekerjaan. Selain karena berhubungan langsung dengan jumlah pendapatan bulanan, pengelompokan ini memiliki rata-rata *relative error* paling kecil yaitu 7,4%. Tetapi spektrum umur yang luas untuk masing-masing jenis pekerjaan menjadi perhatian apakah mereka memiliki distribusi yang luas untuk masing-masing kategori umur atau tidak. Untuk kelompok `'business'` memiliki rentang rata-rata pendapatan bulanan dari 20 ribu hingga 34,9 ribu. Kelompok `'civil servant'` pada rentang 19,5 ribu hingga 34,3 ribu. Kelompok `'employee'` pada rentang 19 ribu hingga 28 ribu. Sedangkan pada pengelompokan kategori usia, distribusi pendapatan tidak begitu melebar yaitu berkisar pada 19,5 ribu hingga 28,7 ribu. Oleh karena itu saya lebih memilih menggunakan gabungan antara kategori usia dan jenis pekerjaan dibandingkan hanya memilih jenis pekerjaan saja.

Alasan yang sama juga berlaku untuk pengelompokan gabungan (jenis pekerjaan, usia dan pendidikan). Distribusi pendapatan untuk antar tingkat pendidikan pada jenis pekerjaan dan usia yang sama memiliki gap yang cukup besar antara nilai tertinggi dan terendahnya. Sehingga untuk memperbaiki nilai yang hilang kita akan menggunakan pengelompokan gabungan antara jenis pekerjaan, kategori usia, dan tingkat pendidikan.

Pengelompokan gabungan (jenis pekerjaan, usia dan pendidikan) memiliki nilai *maximum relative error* yang cukup besar yaitu 39,7%. Akan tetapi nilai *variance relative error* yang cukup kecil pada pengelompokan gabungan (jenis pekerjaan, usia dan pendidikan) yaitu pada nilai 0,54% menandakan hanya sedikit kelompok data pada pengelompokan ini yang memiliki *outlier*. Hal tersebut tercermin pada nilai *mean* dan *median* yang tidak terpaut jauh. Dan oleh karena itu karena terdapat sejumlah kecil kelompok data yang memiliki *outlier* pada pengelompokan ini maka akan lebih baik bila kita menggunakan nilai *median* sebagai acuan pengisian data total pendapatan yang hilang.


In [68]:
#  Membuat fungsi yang akan kita gunakan untuk mengisi nilai yang hilang

def total_income_nanfiller(data:pd.DataFrame)->any:
    reference = incomeType_ageCategory_eduId_income

    try:
        return reference['total_income','median'][data['income_type'],data["age_category"],data['education_id']]
    
    except:
        return "ERROR"

In [69]:
## Membuat dummy dataframe
testing_fillerFunc = pd.DataFrame({
    'income_type':['retiree', 'business', 'employee','student','business','retiree','student'],
    'age_category':['problem','21-25','under 21','21-25','26-30','under 21', '31-35'], 
    'education_id':[1,1,1,0,1,0,0],
    'total_income':[np.nan, 1234, 5678, np.nan,8712,np.nan,np.nan]
    })
testing_fillerFunc


,income_type,age_category,education_id,total_income
0,retiree,problem,1,NaN
1,business,21-25,1,1234.0
2,employee,under 21,1,5678.0
3,student,21-25,0,NaN
4,business,26-30,1,8712.0
5,retiree,under 21,0,NaN
6,student,31-35,0,NaN


In [70]:
## Mengganti nilai yang hilang pada dataframe dummy
testing_fillerFunc.loc[testing_fillerFunc['total_income'].isna(),'total_income'] = testing_fillerFunc[testing_fillerFunc['total_income'].isna()].apply(total_income_nanfiller, axis=1)
testing_fillerFunc

,income_type,age_category,education_id,total_income
0,retiree,problem,1,16419.472
1,business,21-25,1,1234.0
2,employee,under 21,1,5678.0
3,student,21-25,0,15712.26
4,business,26-30,1,8712.0
5,retiree,under 21,0,ERROR
6,student,31-35,0,ERROR


In [71]:
# Menerapkan fungsi tersebut ke setiap baris
scoring.loc[scoring['total_income'].isna(),'total_income'] = scoring[scoring['total_income'].isna()].apply(total_income_nanfiller, axis=1)
scoring.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,0,0,F,employee,0,40620.102,purchase of the house,41-45
1,1,4024.803754,36,1,0,F,employee,0,17932.802,car purchase,36-40
2,0,5623.422610,33,1,0,M,employee,0,23341.752,purchase of the house,31-35
3,3,4124.747207,32,1,0,M,employee,0,42820.568,supplementary education,31-35
4,0,0.000000,53,1,1,F,retiree,0,25378.572,to have a wedding,51-55
5,0,926.185831,27,0,1,M,business,0,40922.17,purchase of the house,26-30
6,0,2879.202052,43,0,0,F,business,0,38484.156,housing transactions,41-45
7,0,152.779569,50,1,0,M,employee,0,21731.829,education,46-50
8,2,6929.865299,35,0,1,F,employee,0,15337.093,having a wedding,31-35
9,0,2188.756445,41,1,0,M,employee,0,23108.15,purchase of the house for my family,41-45


In [72]:
# Memeriksa apakah kita mendapatkan kesalahan
scoring[scoring['total_income']=='ERROR']

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category
8142,0,NaN,64,3,1,F,civil servant,0,ERROR,to have a wedding,61-65


Ditemukan 1 baris data yang memiliki nilai `'total_income'` yaitu `'ERROR'`. Hal tersebut dikarenakan *dataset* referensi untuk pengisian data yang hilang tidak memiliki data median untuk `civil servant` dengan `education_id` bernilai 3 dan berada pada `age_category` bernilai `61-65`.

> Berikut dibawah ini adalah *dataset* referensi untuk `civil servant`, perhatikan pada `age_category` bernilai `61-65` tidak memiliki data untuk `education_id` bernilai 3.


In [73]:
## Melihat kelompok 'civil servant' pada pengelompokkan gabungan: jenis pekerjaan+kategori usia+tingkat pendidikan
incomeType_ageCategory_eduId_income.loc['civil servant']

total_income             abs_rel_error
                                   mean      median              
age_category education_id                                        
21-25        0             29067.471957  24067.2240      0.172022
             1             22426.906593  21361.7300      0.047495
             2             21565.159154  21983.2560      0.019388
26-30        0             27352.900301  25293.8780      0.075276
             1             23372.090506  20825.0300      0.108979
             2             24326.281100  22187.4320      0.087923
             3             30563.383000  30563.3830      0.000000
31-35        0             31456.840484  27952.7200      0.111395
             1             27059.624847  24653.8120      0.088908
             2             31493.758375  29543.3535      0.061930
36-40        0             34052.180453  27959.7690      0.178914
             1             22735.301810  19568.3285      0.139298
             2             30124.717600  27968.5170      0.071576
             3             21150.696000  21150.6960      0.000000
             4             17822.757000  17822.7570      0.000000
41-45        0             31207.645648  28025.0850      0.101980
             1             26003.777660  22349.0400      0.140546
             2             47686.723500  47686.7235      0.000000
46-50        0             36165.875264  29612.8500      0.181194
             1             24227.659703  20871.3990      0.138530
             2             42561.911500  42561.9115      0.000000
             3             78410.774000  78410.7740      0.000000
51-55        0             32796.426583  28489.6200      0.131319
             1             24504.021379  21015.1080      0.142381
56-60        0             29033.394261  26673.4030      0.081285
             1             24592.936019  24220.7485      0.015134
             3             12709.275500  12709.2755      0.000000
61-65        0             38929.432077  42880.3030      0.101488
             1             25225.932208  22148.5830      0.121991
66-70        0             37627.568333  31984.6160      0.149969
             1             31746.657750  26930.8565      0.151695
71-75        1             19530.677500  19530.6775      0.000000
problem      0             25925.315250  23595.8215      0.089854
             1             18696.545500  18696.5455      0.000000
under 21     0             11376.860000  11376.8600      0.000000
             1             39796.052000  39796.0520      0.000000
             2             12125.986000  12125.9860      0.000000

Saat ini *dataset* referensi yang kita gunakan untuk mengisi keseluruhan data yang hilang pada `'total_income'` adalah data pengelompokan gabungan (jenis pekerjaan, kategori usia dan tingkat pendidikan). Oleh karena tidak adanya data yang tepat untuk mengisi 1 buah baris di atas pada *dataset* referensi ini, maka khusus untuk baris tersebut kita akan menggunakan *dataset* referensi yang lain. Adapun *dataset* referensi yang akan kita gunakan adalah data pengelompokan gabungan (jenis pekerjaan dan kategori usia).


In [74]:
## Mengakses data mean dan median pada data pengelompokan gabungan (pekerjaan dan usia) untuk mengisi 1 baris yang bermasalah
incomeType_ageCategory_income.loc['civil servant','61-65']

total_income   mean      30040.675405
               median    23066.187000
abs_rel_error                0.232168
Name: (civil servant, 61-65), dtype: float64

In [75]:
# Mengganti nilai yang bermasalah
scoring.loc[(scoring['income_type']=='civil servant')&(scoring['age_category']=='61-65')&(scoring['education_id']==3),'total_income'] = incomeType_ageCategory_income['total_income','median']['civil servant','61-65']


In [76]:
## Memeriksa data yang baru saja diperbaiki
scoring.loc[(scoring['income_type']=='civil servant')&(scoring['age_category']=='61-65')&(scoring['education_id']==3)]

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category
8142,0,NaN,64,3,1,F,civil servant,0,23066.187,to have a wedding,61-65


**Memeriksa Informasi Pada Data di Kolom `'total_income'`**

In [77]:
# Memeriksa jumlah entri di kolom
scoring.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education_id      21453 non-null  int64  
 4   family_status_id  21453 non-null  int64  
 5   gender            21453 non-null  object 
 6   income_type       21453 non-null  object 
 7   debt              21453 non-null  int64  
 8   total_income      21453 non-null  object 
 9   purpose           21453 non-null  object 
 10  age_category      21453 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 2.0+ MB


In [78]:
## mengubah tipe data yang tidak sesuai pada kolom 'total_income' agar menjadi angka bukan object
scoring['total_income'] = pd.to_numeric(scoring['total_income'])

## Memeriksa kembali tipe data di kolom 'total_income'
scoring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19350 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education_id      21453 non-null  int64  
 4   family_status_id  21453 non-null  int64  
 5   gender            21453 non-null  object 
 6   income_type       21453 non-null  object 
 7   debt              21453 non-null  int64  
 8   total_income      21453 non-null  float64
 9   purpose           21453 non-null  object 
 10  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 2.0+ MB


### <a id='toc2_1_2_'></a>[ Memperbaiki nilai di `days_employed`](#toc0_)

Faktor utama yang mempengaruhi seberapa lama nasabah telah bekerja yaitu usia nasabah itu sendiri. Selain itu jenis pekerjaan ada kemungkinan memiliki faktor yang mempengaruhi nilai seberapa lama nasabah telah bekerja pada pekerjaan tersebut. Seperti pengalaman seseorang yang berumur 50 sebagai `'employee'` sejak berusia 25 tahun tentu akan lebih lama dibandingkan orang yang berumur 50 tahun tetapi baru pensiun (`'retiree'`) beberapa tahun yang lalu.

Pada fase sebelumnya kita telah melakukan sejumlah pengisian nilai bermasalah pada kolom `'days_employed'` ini. Kita menggunakan nilai `0` dan `1` sebagai indikator baris yang bermasalah. Kolom `'days_employed'` bernilai `0` untuk baris dimana nilai `'days_employed'` melebihi usia mereka padahal usia nasabah bukan `0` pada *dataset* awal. Dan nilai `1` pada kolom `'days_employed'` merupakan indikator baris bermasalah yang memiliki nilai `'days_employed'` lebih dari 100 tahun pada *dataset* awal. Oleh karena itu kita perlu melakukan filter pada *dataset* yang telah kita filter dari nilai `NaN`.

In [79]:
## Memfilter dataset tanpa nilai NaN dari nilai 'days_employed' bermasalah
scoring_nonNan_daysEmployed_safe = scoring_non_nan[(scoring_non_nan['days_employed']!=0)&(scoring_non_nan['days_employed']!=1)]
scoring_nonNan_daysEmployed_safe

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,0,0,F,employee,0,40620.102,purchase of the house,41-45
1,1,4024.803754,36,1,0,F,employee,0,17932.802,car purchase,36-40
2,0,5623.422610,33,1,0,M,employee,0,23341.752,purchase of the house,31-35
3,3,4124.747207,32,1,0,M,employee,0,42820.568,supplementary education,31-35
5,0,926.185831,27,0,1,M,business,0,40922.170,purchase of the house,26-30
...,...,...,...,...,...,...,...,...,...,...,...
21519,1,2351.431934,37,4,3,M,employee,0,18551.846,buy commercial real estate,36-40
21520,1,4529.316663,43,1,1,F,business,0,35966.698,housing transactions,41-45
21522,1,2113.346888,38,1,1,M,employee,1,14347.610,property,36-40
21523,3,3112.481705,38,1,0,M,employee,1,39054.888,buying my own car,36-40


In [80]:
# Distribusi median dan rata-rata dari `days_employed` berdasarkan parameter yang telah diidentifikasi

## Faktor Kelompok Usia
age_category_daysEmployed = scoring_nonNan_daysEmployed_safe.groupby('age_category').agg({'days_employed':['mean','median']})
age_category_daysEmployed['abs_rel_error'] = ((age_category_daysEmployed['days_employed','mean']-age_category_daysEmployed['days_employed','median'])/age_category_daysEmployed['days_employed','mean']).abs()
age_category_daysEmployed


days_employed              abs_rel_error
                      mean       median              
age_category                                         
21-25           937.114372   800.559927      0.145718
26-30          1455.499074  1225.819098      0.157802
31-35          1878.202142  1489.511688      0.206948
36-40          2304.673458  1797.225705      0.220182
41-45          2661.766790  1957.465867      0.264599
46-50          2982.226787  2160.231681      0.275631
51-55          3216.225314  2265.592454      0.295574
56-60          3466.150644  2319.817259      0.330722
61-65          3841.536123  2613.341880      0.319714
66-70          3996.495986  2669.073965      0.332146
71-75          3933.895823  2245.558492      0.429177
problem        2200.375775  1560.900431      0.290621
under 21        673.648361   695.968951      0.033134

In [81]:
# Distribusi median dan rata-rata dari `days_employed` berdasarkan parameter yang telah diidentifikasi

## Faktor Jenis Pekerjaan
income_type_daysEmployed = scoring_nonNan_daysEmployed_safe.groupby('income_type').agg({'days_employed':['mean','median']})
income_type_daysEmployed['abs_rel_error'] = ((income_type_daysEmployed['days_employed','mean']-income_type_daysEmployed['days_employed','median'])/income_type_daysEmployed['days_employed','mean']).abs()
income_type_daysEmployed


days_employed              abs_rel_error
                                     mean       median              
income_type                                                         
business                      2111.122879  1545.284206      0.268027
civil servant                 3399.896902  2689.368353      0.208985
employee                      2326.499216  1574.202821      0.323360
paternity / maternity leave   3296.759962  3296.759962      0.000000
student                        578.751554   578.751554      0.000000

In [82]:
# Distribusi median dan rata-rata dari `days_employed` berdasarkan parameter yang telah diidentifikasi

## Faktor Gabungan: Jenis Pekerjaan dan Kategori Usia
incomeType_ageCategory_daysEmployed = scoring_nonNan_daysEmployed_safe.groupby(['income_type','age_category']).agg({'days_employed':['mean','median']})
incomeType_ageCategory_daysEmployed['abs_rel_error'] = ((incomeType_ageCategory_daysEmployed['days_employed','mean']-incomeType_ageCategory_daysEmployed['days_employed','median'])/incomeType_ageCategory_daysEmployed['days_employed','mean']).abs()
incomeType_ageCategory_daysEmployed


days_employed               \
                                                  mean       median   
income_type                 age_category                              
business                    21-25           883.697702   775.242304   
                            26-30          1387.145104  1136.788826   
                            31-35          1752.194093  1407.409093   
                            36-40          2046.017356  1636.482076   
                            41-45          2307.943269  1800.781254   
                            46-50          2736.206319  2045.456606   
                            51-55          2871.786844  1971.004568   
                            56-60          2947.068770  2146.884040   
                            61-65          3683.632143  2358.485102   
                            66-70          3426.511409  1983.280840   
                            71-75          4622.013775  3095.344969   
                            problem        1788.865798  1072.409395   
                            under 21        515.985625   542.867211   
civil servant               21-25          1201.814191  1154.887485   
                            26-30          1877.388339  1778.584171   
                            31-35          2359.068743  2228.577160   
                            36-40          3535.288117  3430.409051   
                            41-45          3860.099898  3221.210688   
                            46-50          4229.410494  3515.898309   
                            51-55          4751.163266  3559.954664   
                            56-60          5245.247752  4195.615025   
                            61-65          4278.099784  3138.344887   
                            66-70          4985.976516  4837.973362   
                            71-75          1204.922098  1204.922098   
                            problem        5878.760924  4945.165547   
                            under 21        600.437357   509.969922   
employee                    21-25           938.604244   797.978948   
                            26-30          1430.123536  1215.333776   
                            31-35          1876.100828  1461.174261   
                            36-40          2267.495822  1716.565270   
                            41-45          2654.021338  1875.723150   
                            46-50          2921.130129  2075.271921   
                            51-55          3168.643480  2241.907558   
                            56-60          3442.480533  2214.334755   
                            61-65          3838.661199  2613.341880   
                            66-70          4075.861669  2830.361431   
                            71-75          4266.205758  4093.896003   
                            problem        1907.113148  1501.480883   
                            under 21        782.125220   868.770587   
paternity / maternity leave 36-40          3296.759962  3296.759962   
student                     21-25           578.751554   578.751554   

                                         abs_rel_error  
                                                        
income_type                 age_category                
business                    21-25             0.122729  
                            26-30             0.180483  
                            31-35             0.196773  
                            36-40             0.200162  
                            41-45             0.219746  
                            46-50             0.252448  
                            51-55             0.313666  
                            56-60             0.271519  
                            61-65             0.359739  
                            66-70             0.421195  
                            71-75             0.330304  
                            problem           0.400509  
                            under 21          0.052098  
civi

In [83]:
## Melihat statistik absolute relative error untuk pengelompokkan kategori jenis pekerjaan
income_type_daysEmployed['abs_rel_error'].agg(['mean','median','min','max','var'])

mean      0.160075
median    0.208985
min       0.000000
max       0.323360
var       0.022989
Name: abs_rel_error, dtype: float64

In [84]:
## Melihat statistik absolute relative error untuk pengelompokkan kategori gabungan: jenis pekerjaan+kategori umur
incomeType_ageCategory_daysEmployed['abs_rel_error'].agg(['mean','median','min','max','var'])

mean      0.192037
median    0.200111
min       0.000000
max       0.421195
var       0.013860
Name: abs_rel_error, dtype: float64

Bila sebelumnya, saya akan menggunakan referensi pengisian data yang hilang menggunakan data pengelompokan gabungan tetapi kini saya akan lebih memilih data pengelompokan kategori usia. Hal tersebut dikarenakan secara umum distribusi `'days_employed'` pada pengelompokan kategori usia dan pengelompokan gabungan (jenis pekerjaan dan usia) memiliki rentang distribusi yang kurang lebih sama. Kemudian ada sedikit kejanggalan pada data pengelompokan gabunagan (jenis pekerjaan dan usia) yaitu pada kelompok `'civil servant'` kategori usia `'71-75'`. Distribusinya sangat berbeda dibandingkan kelompok pekerjaan lainnya, dimana setiap peningkatan kategori usia seharusnya nilai `'days_employed'` juga akan meningkat atau setidaknya pada nilai yang berdekatan dengan kategori usia sebelumnya. Sedangkan pada kelompok `'civil servant'` kategori usia `'71-75'` tersebut, nilai `'days_employed'` malah *drop* layaknya kategori usia `'21-25'`.

>Dan karena nilai rata-rata *absolute relative error* cukup besar pada data *referensi* yang akan kita gunakan (pengelompokan kategori usia) yaitu sebesar 16%, maka kita akan menggunakan data `median` untuk pengisian data yang hilang pada kolom `'days_employed'`.


In [85]:
## Membuat fungsi untuk mengisi nilai 'days_employed' yang hilang
def days_employed_nanfiller(data:pd.DataFrame)->any:
    age_series = data['age_category']
    reference = age_category_daysEmployed
    try:
        return reference['days_employed','median'][age_series]
    
    except:
        return "ERROR"


In [86]:
## Membuat dummy dataframe
testing_fillerFunc = pd.DataFrame({
    'age_category':['problem','21-25','under 21','21-25','26-30','under 21', 'test'],
    'days_employed':[np.nan, 1234, 5678, np.nan,8712,np.nan,np.nan]
    })
testing_fillerFunc



,age_category,days_employed
0,problem,NaN
1,21-25,1234.0
2,under 21,5678.0
3,21-25,NaN
4,26-30,8712.0
5,under 21,NaN
6,test,NaN


In [87]:
## Mengisi data 'days_employed' yang hilang pada data dummy
testing_fillerFunc.loc[testing_fillerFunc['days_employed'].isna(),'days_employed'] = testing_fillerFunc[testing_fillerFunc['days_employed'].isna()].apply(days_employed_nanfiller, axis=1)
testing_fillerFunc

,age_category,days_employed
0,problem,1560.900431
1,21-25,1234.0
2,under 21,5678.0
3,21-25,800.559927
4,26-30,8712.0
5,under 21,695.968951
6,test,ERROR


In [88]:
# Menerapkan fungsi ke days_employed
scoring.loc[scoring['days_employed'].isna(),'days_employed'] = scoring[scoring['days_employed'].isna()].apply(days_employed_nanfiller, axis=1)
scoring.head(20)



,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,0,0,F,employee,0,40620.102,purchase of the house,41-45
1,1,4024.803754,36,1,0,F,employee,0,17932.802,car purchase,36-40
2,0,5623.422610,33,1,0,M,employee,0,23341.752,purchase of the house,31-35
3,3,4124.747207,32,1,0,M,employee,0,42820.568,supplementary education,31-35
4,0,0.000000,53,1,1,F,retiree,0,25378.572,to have a wedding,51-55
5,0,926.185831,27,0,1,M,business,0,40922.170,purchase of the house,26-30
6,0,2879.202052,43,0,0,F,business,0,38484.156,housing transactions,41-45
7,0,152.779569,50,1,0,M,employee,0,21731.829,education,46-50
8,2,6929.865299,35,0,1,F,employee,0,15337.093,having a wedding,31-35
9,0,2188.756445,41,1,0,M,employee,0,23108.150,purchase of the house for my family,41-45


In [89]:
## Memeriksa apakah kita mendapatkan kesalahan
scoring[scoring['days_employed']=='ERROR']

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category


**Memeriksa Informasi Pada Data di Kolom `'days_employed'`**

In [90]:
# Memeriksa entri di semua kolom untuk memastikan kita memperbaiki semua nilai yang hilang
scoring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education_id      21453 non-null  int64  
 4   family_status_id  21453 non-null  int64  
 5   gender            21453 non-null  object 
 6   income_type       21453 non-null  object 
 7   debt              21453 non-null  int64  
 8   total_income      21453 non-null  float64
 9   purpose           21453 non-null  object 
 10  age_category      21453 non-null  object 
dtypes: float64(2), int64(5), object(4)
memory usage: 2.0+ MB


## <a id='toc2_2_'></a>[Pengkategorian Data](#toc0_)



Pengkategorian sangat penting dilakukan pada data-data yang tersebar banyak sehingga perlu dikelompokkan agar dapat melihat pola-pola tertentu. Pengkategorian akan dilakukan untuk tipe data kategorik/teks maupun data kuantitatif/angka. Pada fase sebelumnya kita telah sempat melakukan pengkategorian untuk usia nasabah, sekarang kita akan melanjutkan untuk kolom lainnya. Untuk menentukan karakterisik/kolom mana saja yang akan diberikan pengkategorian, maka kita perlu memperhatikan hipotesis yang ingin diselesaikan. Berikut hipotesisnya.

Adapun hipotesis yang akan diuji pada projek ini adalah sebagai berikut.
1. Jumlah anak yang lebih sedikit dalam keluarga akan meningkatkan kemampuan nasabah dalam melunasi kredit.
2. Nasabah yang belum membina keluarga akan memiliki potensi lebih kecil untuk mengalami gagal bayar dibandingkan nasabah yang telah berkeluarga.
3. Nasabah dengan pendapatan yang lebih kecil memiliki potensi lebih besar untuk gagal bayar.
4. Tujuan atas pengajuan kredit akan menentukan kemampuan pelunasan pinjaman oleh nasabah.

Dari hipotesis yang akan kita selesaikan, karakteristik pendapatan nasabah dan tujuan atas pengajuan kredit akan menjadi bagian yang akan kita kategorikan. Hal tersebut dikarenakan nilai mereka sangat bervariasi dan sulit untuk mengambil keputusan untuk hipotesis. Nilai pendapatan nasabah merupakan variabel kuantitatif yang akan kita kategorikan berdasarkan rentang tertentu. Sedangkan nilai tujuan pengajuan kredit akan menjadi variabel kategorik yang akan kita kelompokkan berdasarkan irisan dari setiap tujuan yang serupa.


In [91]:
## Menampilkan nilai data pada kolom 'purpose'
scoring['purpose'].head()


0      purchase of the house
1               car purchase
2      purchase of the house
3    supplementary education
4          to have a wedding
Name: purpose, dtype: object

In [92]:
## Memeriksa nilai unik pada kolom 'purpose'"Flow Dari Proses Analisis Data.png"
scoring['purpose'].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

Dari *unique value* pada kolom `'purpose'`, dapat dilihat bahwa secara umum mereka dapat dikelompokkan atas objek apa yang ingin mereka peroleh. Dari keseluruhan *unique value* kita dapat mengelompokkannya menjadi 4 kategori berdasarkan objek yang akan diperoleh yaitu:
1. property
2. car
3. wedding
4. education


In [93]:
# Membuat sebuah fungsi untuk mengategorikan data berdasarkan topik umum
def purpose_categorizer(purposeSeries: pd.Series)->any:
    property_list = ['building a property','building a real estate','buy commercial real estate', 'buy real estate','buy residential real estate',
            'buying property for renting out','construction of own property','housing','housing renovation', 'housing transactions',
            'property','purchase of my own house','purchase of the house','purchase of the house for my family','real estate transactions',
            'transactions with commercial real estate','transactions with my real estate']
    
    car_list = ['buying a second-hand car', 'buying my own car','car', 'car purchase','cars','purchase of a car','second-hand car purchase',
            'to buy a car','to own a car']
    
    education_list = ['education','getting an education', 'getting higher education','going to university','profile education','supplementary education',
            'to become educated','to get a supplementary education','university education']
    
    wedding_list = ['having a wedding', 'to have a wedding', 'wedding ceremony']
    
    if purposeSeries in property_list:
        return "property"
    
    elif purposeSeries in car_list:
        return "car"
    
    elif purposeSeries in education_list:
        return "education"
    
    elif purposeSeries in wedding_list:
        return "wedding"

    else:
        return "ERROR"

In [94]:
# Membuat kolom yang memuat kategori
scoring['purpose_category'] = scoring['purpose'].apply(purpose_categorizer)
scoring.head(15)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category
0,1,8437.673028,42,0,0,F,employee,0,40620.102,purchase of the house,41-45,property
1,1,4024.803754,36,1,0,F,employee,0,17932.802,car purchase,36-40,car
2,0,5623.422610,33,1,0,M,employee,0,23341.752,purchase of the house,31-35,property
3,3,4124.747207,32,1,0,M,employee,0,42820.568,supplementary education,31-35,education
4,0,0.000000,53,1,1,F,retiree,0,25378.572,to have a wedding,51-55,wedding
5,0,926.185831,27,0,1,M,business,0,40922.170,purchase of the house,26-30,property
6,0,2879.202052,43,0,0,F,business,0,38484.156,housing transactions,41-45,property
7,0,152.779569,50,1,0,M,employee,0,21731.829,education,46-50,education
8,2,6929.865299,35,0,1,F,employee,0,15337.093,having a wedding,31-35,wedding
9,0,2188.756445,41,1,0,M,employee,0,23108.150,purchase of the house for my family,41-45,property


Untuk data numerik, kita akan melakukan pengelompokan pada kolom `'total_income'`. Hal ini dilakukan untuk menjawab hipotesis No. 3 mengenai hubungan antara tingkat pendapatan bulanan seseorang dengan probabilitas gagal bayar. Kita akan melihat distribusi dari pendapatan bulanan setiap nasabah untuk menentukan akan menggunakan berapa jenis kategori pendapatan serta rentang pendapatan bulanannya.


In [95]:
## Melihat data pada kolom 'total_income'
# print(scoring['total_income'].sort_values().unique())
scoring['total_income'].sort_values()


14585      3306.762
13006      3392.845
16174      3418.824
1598       3471.216
14276      3503.298
            ...    
17178    273809.483
20809    274402.943
9169     276204.162
19606    352136.354
12412    362496.645
Name: total_income, Length: 21453, dtype: float64

In [96]:
# Melihat statistik untuk kolom 'total_income'
scoring['total_income'].describe()


count     21453.00000
mean      26484.89980
std       15733.81348
min        3306.76200
25%       17141.08900
50%       22956.96000
75%       31653.35000
max      362496.64500
Name: total_income, dtype: float64

Pada kolom `'total_income'`, kita akan membagi mereka ke dalam 4 kategori, yaitu nasabah berpendapatan rendah (*`low income`*), berpendapatan menengah ke bawah (*`lower-middle income`*), berpendapatan menengah ke atas (*`upper-middle income`*) dan berpendapatan tinggi (*`high income`*). Terdapat dua cara untuk mengatur pembagian kelompok nasabah berdasarkan pendapatannya, yaitu sebagai berikut. 

1. ***Pertama***, dengan cara mengurutkan nasabah dari pendapatan paling rendah hingga paling tinggi. Kkemudian membagi mereka menjadi 4 kelompok dengan jumlah yang sama setiap kelompoknya. Sehingga bila terdapat 100 sampel nasabah dan telah diurutkan berdasarkan pendapatan dari rendah ke tinggi, maka 25 nasabah pertama merupakan golongan *`low income`*, kemudian 25 nasabah berikutnya merupakan golongan *`lower-middle income`*, dan begitu seterusnya.

2. ***Kedua***, dengan cara mengambil nilai pendapatan tertinggi dan terendah kemudian membaginya menjadi 4 bagian dengan interval gaji yang sama. Sehingga bila pada suatu data ditemukan pendapatan tertinggi dan terendah adalah 100 dan 21 secara berurutan, maka pembagian kelompok menjadi 21-40 (*`low income`*), 41-60 (*`lower-middle income`*), 61-80 (*`upper-middle income`*), dan 81-100 (*`high income`*).

Adapun untuk *dataset* ini saya lebih memilih pembagian kategori berdasarkan cara pertama. Meskipun nilai *`mean`* dan *`median`* tidak terpaut begitu jauh yaitu sebesar 26484.9 dan 22956.96 secara berturut-turut, akan tetapi terdapat distribusi yang kentara yaitu antara ***kuartil 2*** dan ***kuartil 4***. Pada ***kuartil 2***, 25% nasabah pertama memiliki perbedaan *income* yang tidak terpaut jauh satu sama lainnya. Sedangkan pada ***kuartil 4***, 25% nasabah memiliki perbedaan *income* yang sangat jauh jika dibandingkan dengan ***kuartil 2***. Apabila kita menggunakan cara ***Kedua*** dalam mengatur kategori pendapatan nasabah, maka dapat dipastikan **lebih dari 99%** nasabah akan masuk ke kategori 1 yaitu nasabah berpengahasilan rendah (*`low income`*). Oleh karena itu menurut saya metode pembagian kelompok kategori cara ***Pertama*** akan lebih baik.

Untuk mempercepat proses, maka kita akan menggunakan akronim dalam pengkategorian pendapatan nasabah ini, yaitu :
* **L** untuk *`low income`*
* **LM** untuk *`lower-middle income`*
* **UM** untuk *`upper-middle income`*
* **H** untuk *`high income`*


In [97]:
# Membuat fungsi yang melakukan pengkategorian pendapatan nasabah

def income_categorizer(income_series:pd.Series, q1, q2, q3, q4)->any:

    if income_series <= q1:
        return "L"
    
    elif income_series <= q2:
        return "LM"
    
    elif income_series <= q3:
        return "UM"
    
    elif income_series <= q4:
        return "H"
    
    else:
        return "ERROR"


In [98]:
# Membuat kolom pengategorian pendapatan nasabah

q1 = scoring['total_income'].quantile(0.25)
q2 = scoring['total_income'].quantile(0.5)
q3 = scoring['total_income'].quantile(0.75)
q4 = scoring['total_income'].quantile(1)

scoring['income_category'] = scoring['total_income'].apply(income_categorizer, args=(q1,q2,q3,q4))
scoring.sort_values(by='total_income')

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category,income_category
14585,0,0.000000,57,1,0,F,retiree,1,3306.762,property,56-60,property,L
13006,0,0.000000,37,1,1,M,retiree,0,3392.845,going to university,36-40,education,L
16174,1,3642.820023,52,1,0,M,employee,0,3418.824,car purchase,51-55,car,L
1598,0,0.000000,68,1,1,M,retiree,0,3471.216,having a wedding,66-70,wedding,L
14276,0,0.000000,61,1,0,F,retiree,0,3503.298,property,61-65,property,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17178,0,5734.127087,42,0,1,M,business,0,273809.483,to have a wedding,41-45,wedding,H
20809,0,4719.273476,61,1,4,F,employee,0,274402.943,purchase of the house for my family,61-65,property,H
9169,1,5248.554336,35,1,1,M,employee,0,276204.162,supplementary education,31-35,education,H
19606,1,2577.664662,39,0,0,M,business,1,352136.354,building a property,36-40,property,H


In [99]:
# Menghitung setiap nilai kategori untuk melihat pendistribusiannya

scoring['income_category'].value_counts()

L     5364
H     5363
LM    5363
UM    5363
Name: income_category, dtype: int64

## <a id='toc2_3_'></a>[Memeriksa hipotesis](#toc0_)


**Apakah terdapat korelasi antara memiliki anak dengan probabilitas melakukan gagal bayar pinjaman?**

In [100]:
# Memeriksa data anak dan data gagal bayar pinjaman
failed_children = scoring.groupby('children').agg({'children':'count', 'debt':'sum'})
failed_children = failed_children.rename(columns={'children':'count'})
print(failed_children)

# Menghitung persentase gagal bayar berdasarkan jumlah anak
failed_children['failed_ratio'] = failed_children['debt']/failed_children['count']
failed_children.sort_values('failed_ratio')


          count  debt
children             
0         14090  1063
1          4855   445
2          2128   202
3           330    27
4            41     4
5             9     0


,count,debt,failed_ratio
children,,,
5,9,0,0.000000
0,14090,1063,0.075444
3,330,27,0.081818
1,4855,445,0.091658
2,2128,202,0.094925
4,41,4,0.097561


**Kesimpulan**



Secara keseluruhan, rasio antara jumlah nasabah gagal bayar terhadap keseluruhan nasabah pada masing-masing kelompok jumlah anak berada pada rentang 7% hingga 10 %. Dan terdapat satu kelompok yang memiliki rasio bernilai `0`, yaitu kelompok nasabah dengan 5 orang anak. Hal ini wajar dikarenakan jumlah nasabah yang memiliki anak sebanyak 5 anak hanya 9 orang. Sehingga apabila kita menggunakan rasio tertinggi pada rentang yang kita miliki yakni 10% maka untuk memperoleh 1 nasabah yang mengalami gagal bayar pada kelompok 5 anak, maka dibutuhkan sebanyak 10 orang nasabah pada kelompok ini.

Sekarang kita abaikan kelompok nasabah dengan 5 orang anak. Sekarang perhatikan nasabah yang tidak memiliki anak, mereka memiliki rasio paling keci yaitu 7,5% dengan jumlah nasabah sebesar 14090 nasabah yang merupakan kontributor terbesar pada *dataset* ini. Dan apabila kita memperkecil spektrum pengelompokan kita hanya menjadi dua kategori saja yaitu nasabah yang memiliki anak dan tidak memiliki anak, maka dapat dikatakan **nasabah yang tidak memiliki anak akan memiliki probabilitas gagal bayar paling kecil dibandingkan nasabah yang memiliki anak.** Adapun diantara nasabah yang memiliki anak, tidak terlihat perbedaan yang signifikan antara nasabah dengan 1, 2, 3, atau 4 anak dalam probabilitas gagal bayar.


**Apakah terdapat korelasi antara status keluarga dengan probabilitas melakukan gagal bayar pinjaman?**

In [101]:
# Memeriksa data status keluarga dan data gagal bayar pinjaman
failed_family = scoring.groupby('family_status_id').agg({'family_status_id':'count','debt':'sum'})
failed_family = failed_family.rename(columns={'family_status_id':'id_count'})
print(failed_family)

# Menghitung persentase gagal bayar berdasarkan status keluarga
failed_family['failed_ratio'] = failed_family['debt']/failed_family['id_count']

## Menggabungkan data 'failed_family' dengan dictionary family_status_id 'fam_ref' 
failed_family = failed_family.merge(fam_ref,how='left',on='family_status_id').sort_values('failed_ratio')
failed_family.reset_index(drop=True)


                  id_count  debt
family_status_id                
0                    12339   931
1                     4150   388
2                      959    63
3                     1195    85
4                     2810   274


,family_status_id,id_count,debt,failed_ratio,family_status
0,2,959,63,0.065693,widow / widower
1,3,1195,85,0.071130,divorced
2,0,12339,931,0.075452,married
3,1,4150,388,0.093494,civil partnership
4,4,2810,274,0.097509,unmarried


**Kesimpulan**

Dari hasil kalkulasi yang telah dilakukan pada 5 kategori status keluarga nasabah, kelompok para janda/duda (`widow/widower`) memiliki probabilitas paling kecil dibandingkan kelompok lainnya. Probabilitas gagal bayar kelompok janda/duda yaitu 6,6% dan sangat kentara dibandingkan kelompok lainnya. Kemudian kelompok berikutnya yaitu kelompok nasabah yang telah bercerai (`divorced`) dan nasabah dalam status pernikahan (`married`) yang memiliki probabilitas gagal bayar pada kisaran nilai 7%. Kemudian terdapat 2 kelompok terakhir yang memiliki probabilitas gagal bayar yang paling tinggi adalah kelompok nasabah yang memiliki pasangan tanpa ikatan pernikahan (`civil partnership`) dan nasabah yang belum menikah (`unmarried`) dengan probabilitas gagal bayar berkisar pada angka 9%. Sehingga dari 5 kelompok status keluarga setiap nasabah, kita dapat mempersempit kategorinya menjadi 2 kategori umum yaitu:

1. **Nasabah yang pernah menikah/dalam ikatan pernikahan**. Kelompok ini memiliki 2 sub-kategori lainnya yaitu:
    - Nasabah yang ditinggal mati oleh pasangannya
    - Nasabah dengan mantan/pasangan masih hidup
2. **Nasabah yang belum pernah menikah**

Sehingga dapat kita simpulkan bahwa ***nasabah yang belum pernah berada dalam ikatan pernikahan memiliki probabilitas gagal bayar yang lebih tinggi dibandingkan nasabah yang pernah menikah/dalam ikatan pernikahan***. Kemudian dari nasabah yang sudah pernah melakukan pernikahan kita dapat simpulkan pula bahwa ***nasabah yang ditinggal mati oleh pasangannya dan belum menikah kembali memiliki resiko gagal bayar yang lebih rendah dibandingkan nasabah yang sedang dalam ikatan pernikahan atau nasabah yang telah bercerai***.


**Apakah terdapat korelasi antara tingkat pendapatan dengan probabilitas melakukan gagal bayar pinjaman?**

In [102]:
# Memeriksa data tingkat pendapatan dan data gagal bayar pinjaman
failed_income = scoring.groupby('income_category').agg({'income_category':'count', 'debt':'sum'})
failed_income = failed_income.rename(columns={'income_category':'count'})
print(failed_income)


# Menghitung persentase gagal bayar berdasarkan tingkat pendapatan
failed_income['failed_ratio'] = failed_income['debt']/failed_income['count']
failed_income.sort_values('failed_ratio')


                 count  debt
income_category             
H                 5363   375
L                 5364   428
LM                5363   492
UM                5363   446


,count,debt,failed_ratio
income_category,,,
H,5363,375,0.069924
L,5364,428,0.079791
UM,5363,446,0.083162
LM,5363,492,0.091740


**Kesimpulan**


Dari perhitungan yang telah dilakukan terdapat pola yang sangat jelas sekali pada kategori pendapatan nasabah. Tedapat 4 kategori nasabah yaitu berpenghasilan tinggi, menengah-keatas, menengah-kebawah, dan pendapatan rendah. Kelompok nasabah berpengahasilan menengah yaitu kategori `UM` dan `LM` merupakan kelompok yang memiliki probabilitas gagal bayar paling tinggi. Dari kedua kelompok pendapatan menengah tersebut, nasabah yang memiliki pendapatan lebih rendah (`LM`) memiliki probabilitas lebih tinggi untuk gagal bayar dibandingkan kelompok dengan penghasilan lebih tinggi (`UM`). Adapun nasabah dengan probabilitas paling kecil untuk gagal bayar yaitu pada nomor urut satu adalah kelompok *high income* (`H`) dan pada peringkat kedua adalah kelompok *lower income* (`L`).

***Keterangan :***
- `H` *(High Income)*
- `L` *(Low Income)*
- `UM` *(Upper-middle Income)*
- `LM` *(Lower-middle Income)*


Sehingga,

***Kelompok nasabah yang memiliki pendapatan menengah (`UM` dan `LM`) dan rendah (`L`) memiliki probabilitas gagal bayar yang lebih tinggi dibandingkan nasabah dengan penghasilan tinggi (`H`).***


**Bagaimana tujuan kredit memengaruhi persentase gagal bayar?**

In [103]:
# Memeriksa persentase tingkat gagal bayar untuk setiap tujuan kredit dan lakukan penganalisisan
failed_purpose = scoring.groupby('purpose_category').agg({'purpose_category':'count', 'debt':'sum'})
failed_purpose = failed_purpose.rename(columns={'purpose_category':'count'})
failed_purpose['failed_ratio'] = failed_purpose['debt']/failed_purpose['count']

failed_purpose.sort_values('failed_ratio')


,count,debt,failed_ratio
purpose_category,,,
property,10810,782,0.072340
wedding,2324,186,0.080034
education,4013,370,0.092200
car,4306,403,0.093590


**Kesimpulan**


Dari 4 kategori tujuan yang telah kita bentuk, terdapat 2 kategori yang memiliki probabilitas gagal bayar paling tinggi yaitu kategori `education` dan `car` dimana memiliki persentase gagal bayar sebesar 9,2-9,3%. Sedangkan kategori `property` merupakan kategori yang memiliki probabilitas gagal bayar paling kecil yaitu sebesar 7%. Kemudian pada peringkat 2 kategori dengan probabilitas gagal bayar terkecil ditempati oleh kategori `wedding` sebesar 8%.


# <a id='toc3_'></a>[Kesimpulan umum](#toc0_)


*Dataset* ini memiliki 12 kolom dengan 21525 data nasabah. Dari data tersebut ditemukan sejumlah permasalahan sebelum melakukan pengolahan data. Adapun permasalahan yang kita temui pada *dataset* ini yaitu terdapatnya nilai negatif pada jumlah hari lama nasabah bekerja dan jumlah anak. Dan bahkan pada data kolom `'children'` juga ditemukan sejumlah nasabah yang memiliki anak berjumlah 20 orang anak. Selain itu juga terdapat variasi penamaan pada kolom `'education'` untuk *id* yang sama. Selain itu pada kolom tujuan pengajuan kredit terdapat keterangan yang beragam untuk arti yang sama. Selain itu juga ditemukan baris dengan nilai yang hilang sebanyak 2174 data. Kemudian juga ditemukan kelompok nasabah yang memiliki umur bernilai `0`. Selain itu juga ditemukan kelompok nasabah yang memiliki pengalaman kerja melebihi usia mereka dan bahkan juga ditemukan kelompok nasabah yang memiliki pengalaman kerja bernilai ratusan tahun. Selain itu juga ditemukan nilai aneh pada kolom `'gender'` yaitu **'XNA'**.

Penyelesaian atas data yang bermasalah dilakukan dengan sejumlah cara. Pada tahap awal ditemukan bahwa 2174 data yang hilang bersifat simetris, dimana mereka sama-sama memiliki nilai yang hilang pada kolom `'day_employed'` dan `'total_income'`. Disebabkan data yang hilang merupakan 10,1% dari data keseluruhan maka opsi mengganti nilai yang hilang akan lebih baik. Dan setelah dilakukan identifikasi lebih lanjut diperoleh kesimpulan bahwa data yang hilang tidak berkaitan atas karakteristik nasabah tertentu. Hal tersebut dikarenakan rasio antara data yang hilang terhadap data keseluruhan pada masing-masing kelompok karakteristik berada pada kisaran angka yang sama dengan persentase data yang hilang yaitu 10%. Sehingga kehilangan data bersifat merata pada setiap karakteristik diakibatkan `randomness`. Adapun cara pengisian data yang hilang pada kolom `'total_income'` yaitu dengan menggunakan data median pada data(lengkap) yang dikelompokkan berdasarkan jenis pekerjaan, kategori usia, dan tingkat pendidikan. Sedangkan pengisian data yang hilang pada kolom `'days_employed'` yaitu dengan menggunakan data median pada data(lengkap) yang dikelompokkan berdasarkan kategori umur. **Dan data ini akan diisi setelah memperbaiki seluruh permasalahan data yang ada dibawah ini.**

* Kemudian untuk menyeragamkan penulisan variabel kategorik pada kolom `'education'` maka digunakan *style* penulisan *`lower_case`*. Sedangkan nilai negatif pada variabel kuantitatif pada kolom `'children'` dan `'days_employed'` dikonversi agar bernilai positif karena kemungkinan besar diakibatkan kesalahan pada penginputan. Serta pada kolom `'children'` ditemukan sejumlah nasabah yang memiliki anak berjumlah `20`. Saya memutuskan untuk mengkonversi nilai `20` menjadi bernilai `2` karena menganggap ini sebagai kesalahan penginputan. Hal tersebut dikarenakan distribusi yang tidak sesuai untuk pengelompokkan nasabah berdasarkan jumlah anak dimana dsitribusi untuk nasabah yang memiliki anak `0` hingga `5` orang anak menunjukkan penurunan jumlah nasabah. Sementara jumlah nasabah dengan `20` orang anak melebihi jumlah nasabah dengan `4` orang anak.

* Kemudian ditemukan data bermasalah pada sejumlah kolom yaitu pada kolom `'gender'` dimana terdapat nilai *`XNA`* sejumlah 1 nasabah sehingga diputuskan untuk dihapus saja. Kemudian saya juga melakukan penggabungan data antara kelompok pekerjaan `'entrepreneur'` dan `'business'` karena menurut saya itu serupa dan jumlah nasabah `'entrepreneur'` berjumlah jauh sangat kecil dibandingkan `'business'`

* Setelah itu dilakukan pembersihan data dari data-data yang memiliki duplikat ekplisit. 

* Dan untuk permasalahan data yang bermasalah pada `'days_employed'` yang berkenaan dengan nilainya yang melebihi usia nasabah bahkan ada yang bernilai ratusan tahun maka saya memberikan kode angka khusus untuk data ini dengan kode berikut.
    
    * Kolom `'days_employed'` bernilai `0` : Seluruh data yang memiliki nilai `'days_employed'` melebihi usia mereka dan nilai umur nasabah (`'dob_years'`) bukan `0` pada *dataset* awal.
    * Kolom `'days_employed'` bernilai `1` dan kolom `'dob_years'` bernilai `100` : Seluruh data yang memiliki nilai `'days_employed'` lebih dari 100 tahun pada *dataset* awal.
    * Kolom '`dob_years'` bernilai `200` : Seluruh data yang memiliki nilai `'dob_years'` bernilai `0` tetapi `'days_employed'` kurang dari 100 tahun pada *dataset* awal.

Pada tahap pengalohan data, kolom `'purpose'` dikategorikan menjadi lebih kecil yaitu kategori `'car'`, `'education'`,`'property'` dan `'wedding'`. Selain itu juga dilakukan sejumlah pengkategorian lainnya seperti kategori pendapatan nasabah menjadi 4 kategori yaitu *high income (`H`)*, *upper-midde income (`UM`)*, *lower-middle income (`LM`)*, dan *low income (`L`)*.

Dari hasil pengamatan dan perhitungan yang telah dilakukan maka diperoleh hasil akhir yaitu sebagai berikut.
1. **Apakah terdapat hubungan antara memiliki anak dan probabilitas seseorang melakukan gagal bayar pinjaman?**
    
    * Nasabah yang tidak memiliki anak memiliki probabilitas gagal bayar paling kecil yaitu sebesar 7,5%. Sedangkan nasabah yang memiliki anak memiliki resiko gagal bayar lebih besar dengan probabilitas 8% hingga 9,7%.
    
    <br>
2. **Apakah terdapat hubungan antara status perkawinan dan probabilitas seseorang melakukan gagal bayar pinjaman?**

    * Nasabah yang sudah pernah menikah baik yang masih langgeng, bercerai atau ditinggal mati oleh pasangannya memiliki probabilitas gagal bayar yang paling kecil yaitu pada kisaran 6,6% hingga 7,5%. Sedangkan nasabah yang belum pernah melakukan pernikahan memiliki resiko gagal bayar yang lebih besar yaitu pada kisaran 9,3% hingga 9,8%.

    * Dan diantara nasabah yang sudah pernah menikah, nasabah yang ditinggal mati oleh pasangannya memiliki probabilitas gagal bayar lebih kecil yaitu sebesar 6,6%. Sedangkan nasabah yang masih memiliki pasangan/ mantan pasangan masih hidup memiliki resiko gagal bayar lebih besar yaitu berkisar pada angka 7,1% hingga 7,5%.


3. **Apakah terdapat hubungan antara tingkat pendapatan dan probabilitas seseorang melakukan gagal bayar pinjaman?**

    * Nasabah yang memiliki pendapatan pada kategori *`high income`* dengan pendapatan diatas **31653.35** memiliki resiko gagal bayar paling kecil yaitu pada angka 7%. Sedangkan nasabah dari kategori pendapatan menengah dan rendah memiliki resiko gagal bayar paling tinggi yaitu berkisar pada angka 8% hingga 9,2%.

    * Kategori pendapatan menengah (*`lower-middle income`* dan *`upper-middle income`*) dengan pendapatan diantara **17141,1** hingga **31653.35** memiliki resiko gagal bayar paling tinggi yaitu pada kisaran angka 8,3% hingga 9,2%.

    * Dan diantara nasabah kategori pendapatan menengah, nasabah dengan pendapatan *`lower-middle income`* dengan pendapatan diantara **17141,1** hingga **22956.96** memiliki resiko gagal bayar paling tinggi yaitu sebesar 9,2%.

    
4. **Bagaimana perbedaan tujuan pinjaman memengaruhi probabilitas seseorang melakukan gagal bayar pinjaman?**

    * Nasabah yang memiliki tujuan pengajuan kredit berupa perolehan pendidikan dan mobil memiliki resiko gagal bayar paling tinggi yaitu berkisar pada 9,2% hingga 9,35%.
    * Nasabah yang memiliki tujuan untuk perolehan *property* memiliki resiko gagal bayar paling kecil yaitu sebesar 7,2%. Kemudian pada urutan kedua terdapat kelompok nasabah yang memiliki tujuan untuk pernikahan sebesar 8%.
